# Data Cleaning Learning Guide

## What is Data Cleaning?

**Data Cleaning** (also called Data Cleansing or Data Scrubbing) is the process of detecting and correcting (or removing) corrupt, inaccurate, or irrelevant records from a dataset. It's one of the most critical and time-consuming steps in data analysis, often taking 50-80% of a data scientist's time.

## Why is Data Cleaning Important?

1. **Improve Data Quality** - Ensure accuracy, completeness, and reliability
2. **Prevent Errors** - Avoid incorrect analysis and flawed conclusions
3. **Enable Accurate Insights** - Make better decisions with trustworthy data
4. **Boost Model Performance** - Clean data leads to better machine learning results
5. **Save Time Later** - Fix issues early to avoid downstream problems
6. **Meet Standards** - Comply with data quality requirements and regulations
7. **Build Trust** - Stakeholders trust results from clean, validated data

## What You'll Learn in This Notebook

This comprehensive guide covers **15 essential data cleaning topics**:

1. [Creating Sample "Dirty" Data](#1-creating-sample-dirty-data) - Understanding common data quality issues
2. [Handling Missing Values](#2-handling-missing-values) - Detecting and filling gaps
3. [Removing Duplicates](#3-removing-duplicates) - Identifying and eliminating redundancy
4. [Data Type Conversion](#4-data-type-conversion) - Ensuring correct data types
5. [String Cleaning & Standardization](#5-string-cleaning--standardization) - Text data normalization
6. [Handling Outliers & Invalid Values](#6-handling-outliers--invalid-values) - Managing extreme values
7. [Data Validation](#7-data-validation) - Implementing quality checks
8. [Dealing with Inconsistent Data](#8-dealing-with-inconsistent-data) - Standardizing formats
9. [Date & Time Data Cleaning](#9-date--time-data-cleaning) - Standardizing temporal data
10. [Categorical Data Cleaning](#10-categorical-data-cleaning) - Handling categories and labels
11. [Numerical Data Cleaning](#11-numerical-data-cleaning) - Cleaning numeric fields
12. [Text Data Cleaning](#12-text-data-cleaning) - Advanced text processing
13. [Standardization & Normalization](#13-standardization--normalization) - Scaling and transforming
14. [Advanced Data Cleaning Techniques](#14-advanced-data-cleaning-techniques) - Professional methods
15. [Data Cleaning Pipeline & Best Practices](#15-data-cleaning-pipeline--best-practices) - Professional workflows

## Tools & Libraries Used

- **pandas** - Core data manipulation and cleaning
- **numpy** - Numerical operations and array handling
- **re** - Regular expressions for text cleaning
- **datetime** - Date and time parsing and validation
- **matplotlib & seaborn** - Visualization for quality checks

## How to Use This Notebook

1. **Follow sequentially** - Each section builds on previous concepts
2. **Run all examples** - See real cleaning transformations in action
3. **Apply to your data** - Adapt techniques to your own datasets
4. **Document issues** - Keep track of data quality problems found
5. **Create checklists** - Build your own data cleaning workflow

## Common Data Quality Issues

This notebook demonstrates how to handle:

- ❌ **Missing values** - NaN, None, empty strings, placeholders
- ❌ **Duplicates** - Exact and fuzzy duplicate records
- ❌ **Inconsistent formatting** - Mixed case, spacing, date formats
- ❌ **Invalid values** - Out of range, wrong type, impossible values
- ❌ **Typos and errors** - Misspellings, transcription errors
- ❌ **Outliers** - Extreme or anomalous values
- ❌ **Encoding issues** - Special characters, UTF-8 problems
- ❌ **Logical errors** - Violations of business rules

## The Data Cleaning Workflow

```
Raw Data → Inspect → Detect Issues → Clean → Validate → Document → Clean Data
```

1. **Inspect**: Load and examine the data
2. **Detect**: Identify quality issues systematically
3. **Clean**: Apply appropriate cleaning techniques
4. **Validate**: Verify cleaning worked correctly
5. **Document**: Record all cleaning steps taken
6. **Export**: Save cleaned data for analysis

## Data Quality Dimensions

Good data should be:

- ✅ **Accurate** - Values reflect reality
- ✅ **Complete** - All required data is present
- ✅ **Consistent** - No contradictions within data
- ✅ **Timely** - Data is current and up-to-date
- ✅ **Valid** - Values conform to defined formats
- ✅ **Unique** - No unwanted duplicates

---

Let's clean some data! 🧹✨

In [ ]:
# ============================================
# SETUP: Import Libraries for Data Cleaning
# ============================================

# Data manipulation
import pandas as pd
import numpy as np

# Visualization (to see before/after)
import matplotlib.pyplot as plt
import seaborn as sns

# Regular expressions for text cleaning
import re

# Date/time handling
from datetime import datetime, timedelta

# Warning suppression
import warnings
warnings.filterwarnings('ignore')

# Display settings
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
pd.set_option('display.width', None)

print("✓ Libraries imported successfully!")
print(f"Pandas version: {pd.__version__}")
print(f"NumPy version: {np.__version__}")

## 1. Creating Sample "Dirty" Data

**Why**: We need realistic messy data to practice cleaning techniques.

**What makes data "dirty"**:
- Missing values (NaN, None, empty strings)
- Duplicates (exact and fuzzy)
- Inconsistent formatting (case, spacing)
- Invalid values (out of range, wrong type)
- Typos and misspellings
- Extra whitespace
- Mixed data types in same column

In [ ]:
# ============================================
# 1. CREATE SAMPLE "DIRTY" DATA
# ============================================

# Create a messy dataset that mimics real-world data quality issues
dirty_data = {
    'customer_id': [1, 2, 3, 4, 5, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15],
    'name': ['John Doe', 'jane smith', 'ALICE JOHNSON', '  Bob Brown  ', 'Charlie Davis',
             'Charlie Davis', 'Emma Wilson', None, 'Frank Miller', 'Grace Lee', 
             'Henry Taylor', 'Ivy Anderson', 'Jack Thomas', 'Kate Jackson', 'Leo White', 'Mia Harris'],
    'email': ['john@email.com', 'JANE@EMAIL.COM', 'alice@email.com', 'bob@email..com', 
              'charlie@email.com', 'charlie@email.com', '  emma@email.com  ', 'invalid_email',
              'frank@email.com', np.nan, 'henry@email.com', 'ivy@email.com', 
              'jack@email.com', 'kate@email.com', '', 'mia@email.com'],
    'age': [25, 30, '35', 28, 150, 150, 45, 22, -5, 40, 55, np.nan, 33, 29, 38, 42],
    'salary': [50000, 60000, 70000, np.nan, 80000, 80000, 90000, 55000, 
               65000, 75000, 85000, 95000, '$105,000', '115000', 125000, np.nan],
    'city': ['New York', 'new york', 'NEW YORK', 'Los Angeles', 'Chicago',
             'Chicago', 'Houston', 'Phoenix', 'Philadelphia', 'San Antonio',
             'San Diego', None, 'Dallas', 'San Jose', 'Austin', 'Jacksonville'],
    'country': ['USA', 'USA', 'usa', 'USA', 'US', 'US', 'United States', 
                'USA', 'USA', 'USA', 'USA', 'USA', 'U.S.A', 'USA', 'USA', np.nan],
    'phone': ['123-456-7890', '(234) 567-8901', '3456789012', '456.789.0123',
              '567-890-1234', '567-890-1234', '678 901 2345', None, '890-123-4567',
              '901-234-5678', '012-345-6789', '123-456-7890', '234-567-8901',
              '345-678-9012', '456-789-0123', '567-890-1234'],
    'join_date': ['2020-01-15', '2020/02/20', '15-03-2020', '2020.04.10', 
                  '2020-05-05', '2020-05-05', '2020-06-12', '2020-07-18',
                  'Invalid Date', '2020-09-25', '2020-10-30', np.nan,
                  '2020-12-05', '2021-01-10', '2021-02-14', '2021-03-20'],
    'department': ['Sales', 'sales', 'SALES', 'Marketing', 'IT', 'IT', 
                   'HR', 'Finance', 'Sales', 'Marketing', 'IT', 'HR',
                   'Finance', '  Sales  ', 'Marketing', 'IT'],
    'status': ['Active', 'active', 'ACTIVE', 'Inactive', 'Active', 'Active',
               'Inactive', 'Active', '', 'Active', 'Inactive', 'Active',
               'Active', 'Inactive', 'Active', np.nan]
}

df = pd.DataFrame(dirty_data)

print("=" * 80)
print("DIRTY DATA CREATED - INITIAL INSPECTION")
print("=" * 80)

print("\nDataset Shape:", df.shape)
print("\nFirst 10 rows:")
print(df.head(10))

print("\nData Types:")
print(df.dtypes)

print("\nData Info:")
print(df.info())

print("\n⚠️  Data Quality Issues Present:")
print("   ✗ Missing values (NaN, None, empty strings)")
print("   ✗ Duplicate rows")
print("   ✗ Inconsistent case (lowercase, UPPERCASE, Mixed)")
print("   ✗ Extra whitespace")
print("   ✗ Invalid values (negative age, age > 120)")
print("   ✗ Inconsistent formats (dates, phone numbers, country names)")
print("   ✗ Mixed data types (age has string '35')")
print("   ✗ Special characters in numeric fields (salary: '$105,000')")
print("   ✗ Invalid email formats")
print("   ✗ Inconsistent categorical values")

## 2. Handling Missing Values

**Why**: Missing data can bias results, reduce statistical power, and cause errors in analysis.

**When to use**:
- When data has NaN, None, or empty values
- Before any statistical analysis or modeling
- When merging datasets with incomplete records

**Strategies**:
1. **Identify** - Detect all types of missing data
2. **Analyze** - Understand patterns and causes
3. **Decide** - Choose appropriate handling method:
   - **Delete**: Remove rows/columns (if < 5% missing)
   - **Impute**: Fill with mean, median, mode, or predicted values
   - **Flag**: Create indicator for missingness
   - **Keep**: Sometimes missingness is informative

In [ ]:
# ============================================
# 2. HANDLING MISSING VALUES
# ============================================

df_missing = df.copy()

print("=" * 80)
print("MISSING VALUES ANALYSIS & HANDLING")
print("=" * 80)

# 1. Identify missing values (multiple types)
print("\n1. Detecting Missing Values:")
print("\n   Standard NaN/None:")
print(df_missing.isnull().sum())

print("\n   Empty strings:")
empty_strings = (df_missing.select_dtypes(include=['object']) == '').sum()
print(empty_strings[empty_strings > 0])

print("\n   Whitespace-only strings:")
def count_whitespace(series):
    if series.dtype == 'object':
        return series.str.strip().eq('').sum()
    return 0

whitespace_counts = {col: count_whitespace(df_missing[col]) for col in df_missing.select_dtypes(include=['object']).columns}
print({k: v for k, v in whitespace_counts.items() if v > 0})

# 2. Visualize missing data
print("\n2. Missing Data Visualization:")
missing_counts = df_missing.isnull().sum()
missing_percent = (missing_counts / len(df_missing) * 100).sort_values(ascending=False)

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Bar plot
missing_percent[missing_percent > 0].plot(kind='barh', ax=axes[0], color='coral')
axes[0].set_title('Missing Data Percentage by Column')
axes[0].set_xlabel('Percentage Missing (%)')

# Heatmap
sns.heatmap(df_missing.isnull(), yticklabels=False, cbar=True, cmap='viridis', ax=axes[1])
axes[1].set_title('Missing Data Pattern (Yellow = Missing)')

plt.tight_layout()
plt.show()

# 3. Replace empty strings and whitespace with NaN
print("\n3. Standardizing Missing Values (converting empty strings to NaN):")
df_missing = df_missing.replace(r'^\s*$', np.nan, regex=True)
df_missing = df_missing.replace('', np.nan)

print("After standardization:")
print(df_missing.isnull().sum())

# 4. Strategy 1: Drop rows with missing values
print("\n4. Strategy 1: Drop Rows with Any Missing Value")
df_dropped_rows = df_missing.dropna()
print(f"   Before: {len(df_missing)} rows")
print(f"   After: {len(df_dropped_rows)} rows")
print(f"   Rows removed: {len(df_missing) - len(df_dropped_rows)}")

# 5. Strategy 2: Drop columns with too many missing values
print("\n5. Strategy 2: Drop Columns with >50% Missing")
threshold = 0.5
df_dropped_cols = df_missing.loc[:, df_missing.isnull().mean() < threshold]
print(f"   Before: {df_missing.shape[1]} columns")
print(f"   After: {df_dropped_cols.shape[1]} columns")
print(f"   Columns removed: {set(df_missing.columns) - set(df_dropped_cols.columns)}")

# 6. Strategy 3: Impute with statistical measures
print("\n6. Strategy 3: Impute Missing Values")
df_imputed = df_missing.copy()

# Impute numerical columns with median
numerical_cols = df_imputed.select_dtypes(include=[np.number]).columns
for col in numerical_cols:
    median_value = df_imputed[col].median()
    df_imputed[col].fillna(median_value, inplace=True)
    print(f"   {col}: Filled with median = {median_value}")

# Impute categorical columns with mode
categorical_cols = df_imputed.select_dtypes(include=['object']).columns
for col in categorical_cols:
    mode_value = df_imputed[col].mode()[0] if len(df_imputed[col].mode()) > 0 else 'Unknown'
    df_imputed[col].fillna(mode_value, inplace=True)
    print(f"   {col}: Filled with mode = '{mode_value}'")

print(f"\n   Missing values after imputation: {df_imputed.isnull().sum().sum()}")

# 7. Strategy 4: Forward fill / Backward fill (for time series)
print("\n7. Strategy 4: Forward Fill (propagate last valid value)")
df_ffill = df_missing.copy()
df_ffill = df_ffill.fillna(method='ffill')
print(f"   Missing values after forward fill: {df_ffill.isnull().sum().sum()}")

# 8. Strategy 5: Create missing indicator
print("\n8. Strategy 5: Create Missing Indicator Variables")
df_with_indicators = df_missing.copy()

for col in ['salary', 'city', 'country']:
    if col in df_with_indicators.columns:
        indicator_col = f'{col}_missing'
        df_with_indicators[indicator_col] = df_with_indicators[col].isnull().astype(int)
        print(f"   Created: {indicator_col} (1=missing, 0=present)")

print("\nSample with indicators:")
print(df_with_indicators[['salary', 'salary_missing', 'city', 'city_missing']].head())

# 9. Advanced: Impute with interpolation
print("\n9. Advanced: Interpolation for Numeric Data")
df_interpolated = df_missing.copy()

# Convert age to numeric first
df_interpolated['age'] = pd.to_numeric(df_interpolated['age'], errors='coerce')

# Interpolate
df_interpolated['age'] = df_interpolated['age'].interpolate(method='linear')
print(f"   Age missing values after interpolation: {df_interpolated['age'].isnull().sum()}")

# 10. Summary
print("\n10. Missing Value Handling Summary:")
summary = pd.DataFrame({
    'Method': ['Original', 'Drop Rows', 'Drop Columns', 'Impute', 'Forward Fill', 'Interpolate'],
    'Rows': [len(df_missing), len(df_dropped_rows), len(df_dropped_cols), 
             len(df_imputed), len(df_ffill), len(df_interpolated)],
    'Columns': [df_missing.shape[1], df_dropped_rows.shape[1], df_dropped_cols.shape[1],
                df_imputed.shape[1], df_ffill.shape[1], df_interpolated.shape[1]],
    'Missing_Values': [df_missing.isnull().sum().sum(), df_dropped_rows.isnull().sum().sum(),
                      df_dropped_cols.isnull().sum().sum(), df_imputed.isnull().sum().sum(),
                      df_ffill.isnull().sum().sum(), df_interpolated.isnull().sum().sum()]
})
print(summary)

## 3. Removing Duplicates

**Why**: Duplicates distort analysis, inflate counts, and bias statistical measures.

**When to use**:
- When combining data from multiple sources
- After data entry processes
- When unique records are expected
- Before aggregations and analysis

**Types of Duplicates**:
- **Exact duplicates**: All columns identical
- **Partial duplicates**: Key columns identical (e.g., same customer_id)
- **Fuzzy duplicates**: Similar but not identical (e.g., typos)

In [ ]:
# ============================================
# 3. REMOVING DUPLICATES
# ============================================

df_duplicates = df.copy()

print("=" * 80)
print("DUPLICATE DETECTION & REMOVAL")
print("=" * 80)

# 1. Check for exact duplicates (all columns)
print("\n1. Exact Duplicates (All Columns):")
exact_dupes = df_duplicates.duplicated()
print(f"   Number of duplicate rows: {exact_dupes.sum()}")
print(f"   Percentage: {exact_dupes.sum() / len(df_duplicates) * 100:.1f}%")

if exact_dupes.sum() > 0:
    print("\n   Duplicate rows:")
    print(df_duplicates[exact_dupes])

# 2. Check duplicates on specific columns (e.g., customer_id)
print("\n2. Duplicates Based on Key Column (customer_id):")
key_dupes = df_duplicates.duplicated(subset=['customer_id'])
print(f"   Duplicate customer_ids: {key_dupes.sum()}")

if key_dupes.sum() > 0:
    print("\n   Records with duplicate customer_ids:")
    duplicate_ids = df_duplicates[df_duplicates.duplicated(subset=['customer_id'], keep=False)]
    print(duplicate_ids.sort_values('customer_id'))

# 3. View all occurrences of duplicates (including first)
print("\n3. All Occurrences of Duplicates (including first):")
all_dupes = df_duplicates[df_duplicates.duplicated(subset=['customer_id'], keep=False)]
print(all_dupes.sort_values('customer_id'))

# 4. Remove exact duplicates - keep first occurrence
print("\n4. Remove Exact Duplicates (keep='first'):")
df_no_dupes_first = df_duplicates.drop_duplicates()
print(f"   Before: {len(df_duplicates)} rows")
print(f"   After: {len(df_no_dupes_first)} rows")
print(f"   Removed: {len(df_duplicates) - len(df_no_dupes_first)} rows")

# 5. Remove duplicates - keep last occurrence
print("\n5. Remove Duplicates (keep='last'):")
df_no_dupes_last = df_duplicates.drop_duplicates(keep='last')
print(f"   After (keep last): {len(df_no_dupes_last)} rows")

# 6. Remove duplicates based on specific columns
print("\n6. Remove Duplicates Based on customer_id:")
df_no_dupes_id = df_duplicates.drop_duplicates(subset=['customer_id'], keep='first')
print(f"   Before: {len(df_duplicates)} rows")
print(f"   After: {len(df_no_dupes_id)} rows")
print(f"   Removed: {len(df_duplicates) - len(df_no_dupes_id)} rows")

# 7. Remove all occurrences of duplicates (keep=False)
print("\n7. Remove ALL Occurrences (keep=False):")
df_remove_all = df_duplicates.drop_duplicates(subset=['customer_id'], keep=False)
print(f"   After (removed all duplicate instances): {len(df_remove_all)} rows")

# 8. Identify duplicate groups
print("\n8. Identify Duplicate Groups:")
duplicate_groups = df_duplicates.groupby('customer_id').filter(lambda x: len(x) > 1)
print(f"   Number of records in duplicate groups: {len(duplicate_groups)}")
print("\n   Duplicate groups:")
print(duplicate_groups.sort_values('customer_id'))

# 9. Custom duplicate detection (case-insensitive email)
print("\n9. Custom Duplicate Detection (case-insensitive email):")
df_custom = df_duplicates.copy()
df_custom['email_lower'] = df_custom['email'].str.lower().str.strip()
email_dupes = df_custom.duplicated(subset=['email_lower'], keep=False)
print(f"   Duplicate emails (case-insensitive): {email_dupes.sum()}")

if email_dupes.sum() > 0:
    print("\n   Records with duplicate emails:")
    print(df_custom[email_dupes][['customer_id', 'name', 'email', 'email_lower']].sort_values('email_lower'))

# 10. Fuzzy duplicate detection (name similarity)
print("\n10. Fuzzy Duplicate Detection (similar names):")

# Simple approach: check for names that differ only in whitespace or case
df_fuzzy = df_duplicates.copy()
df_fuzzy['name_normalized'] = df_fuzzy['name'].str.lower().str.strip().str.replace(r'\s+', ' ', regex=True)
fuzzy_dupes = df_fuzzy.duplicated(subset=['name_normalized'], keep=False)

print(f"   Similar names found: {fuzzy_dupes.sum()}")
if fuzzy_dupes.sum() > 0:
    print("\n   Records with similar names:")
    print(df_fuzzy[fuzzy_dupes][['customer_id', 'name', 'name_normalized']].sort_values('name_normalized'))

# 11. Remove duplicates and reset index
print("\n11. Remove Duplicates and Reset Index:")
df_clean = df_duplicates.drop_duplicates(subset=['customer_id']).reset_index(drop=True)
print(f"   Shape after cleanup: {df_clean.shape}")
print("\n   Clean data (first 5 rows):")
print(df_clean.head())

# 12. Summary
print("\n12. Duplicate Removal Summary:")
summary = pd.DataFrame({
    'Method': [
        'Original',
        'Remove exact duplicates (all columns)',
        'Remove duplicates by customer_id',
        'Remove all duplicate occurrences',
        'Custom (case-insensitive email)'
    ],
    'Rows': [
        len(df_duplicates),
        len(df_no_dupes_first),
        len(df_no_dupes_id),
        len(df_remove_all),
        len(df_custom.drop_duplicates(subset=['email_lower']))
    ],
    'Rows_Removed': [
        0,
        len(df_duplicates) - len(df_no_dupes_first),
        len(df_duplicates) - len(df_no_dupes_id),
        len(df_duplicates) - len(df_remove_all),
        len(df_custom) - len(df_custom.drop_duplicates(subset=['email_lower']))
    ]
})
print(summary.to_string(index=False))

## 4. Data Type Conversion

**Why**: Correct data types ensure proper operations, storage efficiency, and prevent errors.

**When to use**:
- After loading data (often defaults to object/string)
- Before mathematical operations
- Before date/time manipulations
- To optimize memory usage

**Common Conversions**:
- String → Numeric (int, float)
- String → DateTime
- Numeric → Category (for memory efficiency)
- Object → Boolean

In [ ]:
# ============================================
# 4. DATA TYPE CONVERSION
# ============================================

df_types = df.drop_duplicates(subset=['customer_id']).reset_index(drop=True).copy()

print("=" * 80)
print("DATA TYPE CONVERSION")
print("=" * 80)

# 1. Check current data types
print("\n1. Current Data Types:")
print(df_types.dtypes)

print("\n   Memory usage before conversion:")
print(f"   Total: {df_types.memory_usage(deep=True).sum() / 1024:.2f} KB")

# 2. Convert age to numeric (handling errors)
print("\n2. Converting 'age' to Numeric:")
print(f"   Before: {df_types['age'].dtype}")
print(f"   Sample values: {df_types['age'].head()}")

# errors='coerce' converts invalid values to NaN
df_types['age'] = pd.to_numeric(df_types['age'], errors='coerce')
print(f"   After: {df_types['age'].dtype}")
print(f"   NaN values created: {df_types['age'].isnull().sum()}")

# 3. Clean and convert salary
print("\n3. Converting 'salary' to Numeric:")
print(f"   Before: {df_types['salary'].dtype}")
print(f"   Sample problematic value: {df_types[df_types['salary'].astype(str).str.contains('\\$', na=False)]['salary'].values}")

# Remove $, commas, and convert to numeric
df_types['salary_clean'] = df_types['salary'].astype(str).str.replace(r'[\$,]', '', regex=True)
df_types['salary'] = pd.to_numeric(df_types['salary_clean'], errors='coerce')
df_types = df_types.drop('salary_clean', axis=1)

print(f"   After: {df_types['salary'].dtype}")
print(f"   Sample values: {df_types['salary'].head()}")

# 4. Convert categorical columns to category type
print("\n4. Converting to Category Type (memory optimization):")
categorical_columns = ['city', 'country', 'department', 'status']

for col in categorical_columns:
    if col in df_types.columns:
        memory_before = df_types[col].memory_usage(deep=True)
        df_types[col] = df_types[col].astype('category')
        memory_after = df_types[col].memory_usage(deep=True)
        print(f"   {col}: {memory_before} → {memory_after} bytes (saved {memory_before - memory_after} bytes)")

# 5. Convert join_date to datetime
print("\n5. Converting 'join_date' to DateTime:")
print(f"   Before: {df_types['join_date'].dtype}")
print(f"   Sample values: {df_types['join_date'].head()}")

# Parse multiple date formats
df_types['join_date'] = pd.to_datetime(df_types['join_date'], errors='coerce', 
                                       infer_datetime_format=True)
print(f"   After: {df_types['join_date'].dtype}")
print(f"   Invalid dates converted to NaT: {df_types['join_date'].isnull().sum()}")

# 6. Extract date components
print("\n6. Extracting Date Components:")
df_types['join_year'] = df_types['join_date'].dt.year
df_types['join_month'] = df_types['join_date'].dt.month
df_types['join_day'] = df_types['join_date'].dt.day
df_types['join_dayofweek'] = df_types['join_date'].dt.day_name()

print("   New columns created:")
print(df_types[['join_date', 'join_year', 'join_month', 'join_day', 'join_dayofweek']].head())

# 7. Convert to boolean
print("\n7. Creating Boolean Column:")
df_types['is_active'] = df_types['status'].str.lower() == 'active'
print("   Status → is_active:")
print(df_types[['status', 'is_active']].head())

# 8. Downcast numeric types to save memory
print("\n8. Downcasting Numeric Types:")
print("   Before downcasting:")
print(df_types[['customer_id', 'age']].dtypes)

# Downcast to smallest suitable type
df_types['customer_id'] = pd.to_numeric(df_types['customer_id'], downcast='integer')
df_types['age'] = pd.to_numeric(df_types['age'], downcast='integer')

print("   After downcasting:")
print(df_types[['customer_id', 'age']].dtypes)

# 9. Convert object to string (explicit)
print("\n9. Converting Object to String (explicit):")
string_cols = ['name', 'email', 'phone']
for col in string_cols:
    if col in df_types.columns and df_types[col].dtype == 'object':
        df_types[col] = df_types[col].astype('string')
        print(f"   {col}: object → string")

# 10. Summary after conversion
print("\n10. Data Types After Conversion:")
print(df_types.dtypes)

print("\n    Memory usage after conversion:")
print(f"    Total: {df_types.memory_usage(deep=True).sum() / 1024:.2f} KB")

# 11. Type conversion with custom function
print("\n11. Custom Type Conversion Function:")

def convert_to_proper_types(df):
    """
    Automatically detect and convert data types
    """
    df_converted = df.copy()
    
    for col in df_converted.columns:
        # Try numeric conversion
        if df_converted[col].dtype == 'object':
            # Try converting to numeric
            try:
                numeric_col = pd.to_numeric(df_converted[col], errors='coerce')
                if numeric_col.notna().sum() / len(df_converted) > 0.5:  # If >50% successfully converted
                    df_converted[col] = numeric_col
                    continue
            except:
                pass
            
            # Try datetime conversion
            try:
                datetime_col = pd.to_datetime(df_converted[col], errors='coerce')
                if datetime_col.notna().sum() / len(df_converted) > 0.5:
                    df_converted[col] = datetime_col
                    continue
            except:
                pass
            
            # Convert low-cardinality strings to category
            if df_converted[col].nunique() / len(df_converted) < 0.5:
                df_converted[col] = df_converted[col].astype('category')
    
    return df_converted

print("   Function created: convert_to_proper_types()")
print("   Use this to automatically detect and convert types!")

# 12. Data type validation
print("\n12. Type Validation:")
expected_types = {
    'customer_id': ['int8', 'int16', 'int32', 'int64'],
    'age': ['int8', 'int16', 'int32', 'int64', 'float64'],
    'salary': ['float64'],
    'join_date': ['datetime64[ns]'],
    'department': ['category']
}

print("   Validating expected types:")
for col, expected in expected_types.items():
    if col in df_types.columns:
        actual = str(df_types[col].dtype)
        status = "✓" if actual in expected else "✗"
        print(f"   {status} {col}: expected {expected}, got {actual}")

## 5. String Cleaning & Standardization

**Why**: Inconsistent text causes duplicate detection failures and analysis errors.

**When to use**:
- Before grouping or aggregating
- When matching records
- Before categorical encoding
- When cleaning user-entered data

**Common Issues**:
- Inconsistent case (John vs JOHN vs john)
- Extra whitespace
- Special characters
- Typos and misspellings
- Inconsistent formatting

In [ ]:
# ============================================
# 5. STRING CLEANING & STANDARDIZATION
# ============================================

df_strings = df.drop_duplicates(subset=['customer_id']).reset_index(drop=True).copy()

print("=" * 80)
print("STRING CLEANING & STANDARDIZATION")
print("=" * 80)

# 1. Remove leading/trailing whitespace
print("\n1. Removing Leading/Trailing Whitespace:")
print("   Before:")
print(df_strings[['name', 'email', 'city']].head())

df_strings['name'] = df_strings['name'].str.strip()
df_strings['email'] = df_strings['email'].str.strip()
df_strings['city'] = df_strings['city'].str.strip()
df_strings['department'] = df_strings['department'].str.strip()

print("\n   After strip():")
print(df_strings[['name', 'email', 'city']].head())

# 2. Standardize case
print("\n2. Standardizing Case:")
print("   Original cities:", df_strings['city'].unique())

# Title case for names
df_strings['name_clean'] = df_strings['name'].str.title()

# Lowercase for emails
df_strings['email_clean'] = df_strings['email'].str.lower()

# Title case for cities
df_strings['city_clean'] = df_strings['city'].str.title()

# Uppercase for country codes
df_strings['country_clean'] = df_strings['country'].str.upper()

print("\n   Cleaned:")
print(df_strings[['name', 'name_clean', 'city', 'city_clean']].head())

# 3. Remove extra spaces between words
print("\n3. Removing Extra Spaces:")
sample_text = "  John    Doe  "
print(f"   Before: '{sample_text}'")
cleaned = re.sub(r'\s+', ' ', sample_text).strip()
print(f"   After: '{cleaned}'")

df_strings['name_clean'] = df_strings['name_clean'].str.replace(r'\s+', ' ', regex=True)
print("\n   Applied to names:")
print(df_strings[['name', 'name_clean']].head())

# 4. Replace special characters
print("\n4. Replacing Special Characters:")

# Replace dots in email domains
print("   Before (email):", df_strings['email'].iloc[3])
df_strings['email_clean'] = df_strings['email_clean'].str.replace(r'\.\.+', '.', regex=True)
print("   After (fixed double dots):", df_strings['email_clean'].iloc[3])

# Remove special characters from phone
print("\n   Standardizing phone numbers:")
print("   Before:", df_strings['phone'].head())

df_strings['phone_clean'] = df_strings['phone'].str.replace(r'[^0-9]', '', regex=True)
print("   After (digits only):", df_strings['phone_clean'].head())

# 5. Standardize categorical values
print("\n5. Standardizing Categorical Values:")

# Create mapping for country variations
country_mapping = {
    'USA': 'USA',
    'US': 'USA',
    'U.S.A': 'USA',
    'UNITED STATES': 'USA'
}

print("   Before:", df_strings['country'].value_counts())

df_strings['country_clean'] = df_strings['country'].str.upper().map(country_mapping)
df_strings['country_clean'] = df_strings['country_clean'].fillna(df_strings['country'].str.upper())

print("\n   After mapping:")
print(df_strings['country_clean'].value_counts())

# 6. Fix typos and common misspellings
print("\n6. Fixing Common Misspellings:")

# Department standardization
dept_mapping = {
    'SALES': 'Sales',
    'MARKETING': 'Marketing',
    'IT': 'IT',
    'HR': 'HR',
    'FINANCE': 'Finance'
}

df_strings['department_clean'] = df_strings['department'].str.upper().map(dept_mapping)
print("   Department values after standardization:")
print(df_strings['department_clean'].value_counts())

# 7. Validate email format
print("\n7. Validating Email Format:")

def is_valid_email(email):
    """Check if email format is valid"""
    if pd.isna(email):
        return False
    pattern = r'^[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}$'
    return bool(re.match(pattern, str(email)))

df_strings['email_valid'] = df_strings['email_clean'].apply(is_valid_email)

print(f"   Valid emails: {df_strings['email_valid'].sum()}")
print(f"   Invalid emails: {(~df_strings['email_valid']).sum()}")

print("\n   Invalid email examples:")
print(df_strings[~df_strings['email_valid']][['name_clean', 'email', 'email_clean']])

# 8. Standardize phone format
print("\n8. Standardizing Phone Format:")

def format_phone(phone):
    """Format phone number as XXX-XXX-XXXX"""
    if pd.isna(phone):
        return None
    
    # Remove all non-digits
    digits = re.sub(r'[^0-9]', '', str(phone))
    
    # Format if 10 digits
    if len(digits) == 10:
        return f"{digits[:3]}-{digits[3:6]}-{digits[6:]}"
    else:
        return digits

df_strings['phone_formatted'] = df_strings['phone'].apply(format_phone)

print("   Before → After:")
for i in range(min(5, len(df_strings))):
    print(f"   {df_strings['phone'].iloc[i]} → {df_strings['phone_formatted'].iloc[i]}")

# 9. Remove non-ASCII characters
print("\n9. Removing Non-ASCII Characters:")

sample_with_unicode = "José García™"
print(f"   Before: {sample_with_unicode}")

cleaned_ascii = sample_with_unicode.encode('ascii', 'ignore').decode('ascii')
print(f"   After: {cleaned_ascii}")

# Apply to name column
df_strings['name_ascii'] = df_strings['name_clean'].str.encode('ascii', 'ignore').str.decode('ascii')

# 10. Consistent capitalization for abbreviations
print("\n10. Handling Abbreviations:")

def standardize_abbreviations(text):
    """Standardize common abbreviations"""
    if pd.isna(text):
        return text
    
    # Replace common abbreviations
    replacements = {
        r'\bSt\b': 'Street',
        r'\bAve\b': 'Avenue',
        r'\bDr\b': 'Drive',
        r'\bRd\b': 'Road',
        r'\bBlvd\b': 'Boulevard'
    }
    
    result = str(text)
    for pattern, replacement in replacements.items():
        result = re.sub(pattern, replacement, result, flags=re.IGNORECASE)
    
    return result

# Example usage
address = "123 Main St, Apt 4B"
print(f"   Before: {address}")
print(f"   After: {standardize_abbreviations(address)}")

# 11. Comprehensive string cleaning function
print("\n11. Comprehensive String Cleaning Function:")

def clean_string(text, 
                 strip=True, 
                 lower=False, 
                 upper=False, 
                 title=False,
                 remove_special=False,
                 remove_digits=False,
                 remove_extra_spaces=True):
    """
    Comprehensive string cleaning function
    
    Parameters:
    -----------
    text : str
        Text to clean
    strip : bool
        Remove leading/trailing whitespace
    lower : bool
        Convert to lowercase
    upper : bool
        Convert to uppercase
    title : bool
        Convert to title case
    remove_special : bool
        Remove special characters
    remove_digits : bool
        Remove digits
    remove_extra_spaces : bool
        Replace multiple spaces with single space
    
    Returns:
    --------
    str : Cleaned text
    """
    if pd.isna(text):
        return text
    
    result = str(text)
    
    if strip:
        result = result.strip()
    
    if remove_special:
        result = re.sub(r'[^a-zA-Z0-9\s]', '', result)
    
    if remove_digits:
        result = re.sub(r'\d', '', result)
    
    if remove_extra_spaces:
        result = re.sub(r'\s+', ' ', result)
    
    if lower:
        result = result.lower()
    elif upper:
        result = result.upper()
    elif title:
        result = result.title()
    
    return result

# Test the function
test_text = "  John@123  DOE!!  "
print(f"   Original: '{test_text}'")
print(f"   Cleaned: '{clean_string(test_text, title=True, remove_special=True, remove_digits=True)}'")

# 12. Apply comprehensive cleaning
print("\n12. Final Cleaned Data Sample:")
final_clean_cols = ['name_clean', 'email_clean', 'city_clean', 'country_clean', 
                    'phone_formatted', 'department_clean']

print(df_strings[final_clean_cols].head(10))

# 13. Summary of string cleaning operations
print("\n13. String Cleaning Summary:")
print("   ✓ Whitespace removed (leading/trailing)")
print("   ✓ Case standardized (title/lower/upper as appropriate)")
print("   ✓ Extra spaces removed")
print("   ✓ Special characters handled")
print("   ✓ Phone numbers formatted consistently")
print("   ✓ Email validation performed")
print("   ✓ Categorical values standardized")
print("   ✓ Country codes unified")
print("   ✓ Department names standardized")

## 6. Handling Outliers & Invalid Values

**Why**: Outliers and invalid values can skew analysis and break models.

**When to use**:
- Before statistical analysis
- When preparing data for modeling
- After data integration
- When detecting data entry errors

**Approaches**:
- **Detect**: IQR, Z-score, domain knowledge
- **Handle**: Remove, cap, transform, or keep (if legitimate)
- **Validate**: Check against business rules

In [ ]:
# ============================================
# 6. HANDLING OUTLIERS & INVALID VALUES
# ============================================

df_outliers = df.drop_duplicates(subset=['customer_id']).reset_index(drop=True).copy()

# Convert age and salary to numeric
df_outliers['age'] = pd.to_numeric(df_outliers['age'], errors='coerce')
df_outliers['salary'] = df_outliers['salary'].astype(str).str.replace(r'[\$,]', '', regex=True)
df_outliers['salary'] = pd.to_numeric(df_outliers['salary'], errors='coerce')

print("=" * 80)
print("HANDLING OUTLIERS & INVALID VALUES")
print("=" * 80)

# 1. Identify invalid values (domain knowledge)
print("\n1. Identifying Invalid Values (Business Rules):")

# Age should be between 18 and 100
print("\n   Age validation (valid range: 18-100):")
invalid_age = (df_outliers['age'] < 18) | (df_outliers['age'] > 100)
print(f"   Invalid ages: {invalid_age.sum()}")
if invalid_age.sum() > 0:
    print(df_outliers[invalid_age][['customer_id', 'name', 'age']])

# Salary should be positive and reasonable (e.g., 10000-500000)
print("\n   Salary validation (valid range: $10,000-$500,000):")
invalid_salary = (df_outliers['salary'] < 10000) | (df_outliers['salary'] > 500000)
print(f"   Invalid salaries: {invalid_salary.sum()}")
if invalid_salary.sum() > 0:
    print(df_outliers[invalid_salary][['customer_id', 'name', 'salary']])

# 2. Detect outliers using IQR method
print("\n2. Detecting Outliers (IQR Method):")

def detect_outliers_iqr(df, column):
    """Detect outliers using IQR method"""
    Q1 = df[column].quantile(0.25)
    Q3 = df[column].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    
    outliers = (df[column] < lower_bound) | (df[column] > upper_bound)
    
    return outliers, lower_bound, upper_bound

# Age outliers
age_outliers, age_lower, age_upper = detect_outliers_iqr(df_outliers, 'age')
print(f"\n   Age outliers (IQR method):")
print(f"   Lower bound: {age_lower:.1f}, Upper bound: {age_upper:.1f}")
print(f"   Outliers found: {age_outliers.sum()}")

# Salary outliers
salary_outliers, salary_lower, salary_upper = detect_outliers_iqr(df_outliers, 'salary')
print(f"\n   Salary outliers (IQR method):")
print(f"   Lower bound: ${salary_lower:,.0f}, Upper bound: ${salary_upper:,.0f}")
print(f"   Outliers found: {salary_outliers.sum()}")

# 3. Visualize outliers
print("\n3. Visualizing Outliers:")

fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# Age boxplot
df_outliers.boxplot(column='age', ax=axes[0, 0])
axes[0, 0].set_title('Age Distribution (Box Plot)')
axes[0, 0].set_ylabel('Age')
axes[0, 0].axhline(y=age_lower, color='r', linestyle='--', label='Lower bound')
axes[0, 0].axhline(y=age_upper, color='r', linestyle='--', label='Upper bound')
axes[0, 0].legend()

# Age histogram
df_outliers['age'].hist(bins=20, ax=axes[0, 1], edgecolor='black')
axes[0, 1].set_title('Age Distribution (Histogram)')
axes[0, 1].set_xlabel('Age')
axes[0, 1].set_ylabel('Frequency')

# Salary boxplot
df_outliers.boxplot(column='salary', ax=axes[1, 0])
axes[1, 0].set_title('Salary Distribution (Box Plot)')
axes[1, 0].set_ylabel('Salary')

# Salary histogram
df_outliers['salary'].hist(bins=20, ax=axes[1, 1], edgecolor='black')
axes[1, 1].set_title('Salary Distribution (Histogram)')
axes[1, 1].set_xlabel('Salary')
axes[1, 1].set_ylabel('Frequency')

plt.tight_layout()
plt.show()

# 4. Handle invalid values - Method 1: Remove
print("\n4. Method 1: Remove Invalid Values")
df_removed = df_outliers.copy()

# Remove invalid ages
df_removed = df_removed[(df_removed['age'] >= 18) & (df_removed['age'] <= 100)]
print(f"   Rows after removing invalid ages: {len(df_removed)}")

# Remove invalid salaries
df_removed = df_removed[(df_removed['salary'] >= 10000) & (df_removed['salary'] <= 500000)]
print(f"   Rows after removing invalid salaries: {len(df_removed)}")

# 5. Handle outliers - Method 2: Cap (Winsorization)
print("\n5. Method 2: Cap Outliers (Winsorization)")
df_capped = df_outliers.copy()

# Cap age at valid limits
df_capped['age_capped'] = df_capped['age'].clip(lower=18, upper=100)
print(f"   Age: min={df_capped['age_capped'].min():.0f}, max={df_capped['age_capped'].max():.0f}")

# Cap salary at IQR bounds
df_capped['salary_capped'] = df_capped['salary'].clip(lower=salary_lower, upper=salary_upper)
print(f"   Salary: min=${df_capped['salary_capped'].min():,.0f}, max=${df_capped['salary_capped'].max():,.0f}")

# 6. Handle outliers - Method 3: Transform
print("\n6. Method 3: Transform Data (Log Transformation)")
df_transformed = df_outliers.copy()

# Log transform salary to reduce impact of outliers
df_transformed['salary_log'] = np.log1p(df_transformed['salary'])

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

df_outliers['salary'].hist(bins=20, ax=axes[0], edgecolor='black')
axes[0].set_title('Original Salary Distribution')
axes[0].set_xlabel('Salary')

df_transformed['salary_log'].hist(bins=20, ax=axes[1], edgecolor='black', color='green')
axes[1].set_title('Log-Transformed Salary')
axes[1].set_xlabel('Log(Salary)')

plt.tight_layout()
plt.show()

# 7. Handle outliers - Method 4: Replace with median
print("\n7. Method 4: Replace Outliers with Median")
df_replaced = df_outliers.copy()

# Replace age outliers with median
age_median = df_replaced['age'].median()
df_replaced.loc[age_outliers, 'age'] = age_median
print(f"   Age outliers replaced with median: {age_median:.0f}")

# Replace salary outliers with median
salary_median = df_replaced['salary'].median()
df_replaced.loc[salary_outliers, 'salary'] = salary_median
print(f"   Salary outliers replaced with median: ${salary_median:,.0f}")

# 8. Create outlier flags
print("\n8. Creating Outlier Indicator Flags")
df_flagged = df_outliers.copy()

df_flagged['age_outlier'] = age_outliers.astype(int)
df_flagged['salary_outlier'] = salary_outliers.astype(int)

print("   Sample with flags:")
print(df_flagged[['name', 'age', 'age_outlier', 'salary', 'salary_outlier']].head(10))

# 9. Validate against multiple rules
print("\n9. Multi-Rule Validation")

def validate_record(row):
    """Validate a record against multiple business rules"""
    issues = []
    
    # Age validation
    if pd.notna(row['age']):
        if row['age'] < 18:
            issues.append('Age too young')
        elif row['age'] > 100:
            issues.append('Age too old')
    
    # Salary validation
    if pd.notna(row['salary']):
        if row['salary'] < 10000:
            issues.append('Salary too low')
        elif row['salary'] > 500000:
            issues.append('Salary too high')
    
    # Email validation
    if pd.notna(row['email']):
        if '@' not in str(row['email']):
            issues.append('Invalid email format')
    
    return '; '.join(issues) if issues else 'Valid'

df_outliers['validation_status'] = df_outliers.apply(validate_record, axis=1)

print("   Validation results:")
print(df_outliers['validation_status'].value_counts())

print("\n   Records with issues:")
invalid_records = df_outliers[df_outliers['validation_status'] != 'Valid']
print(invalid_records[['customer_id', 'name', 'age', 'salary', 'email', 'validation_status']])

# 10. Summary of outlier handling methods
print("\n10. Outlier Handling Methods Summary:")
summary = pd.DataFrame({
    'Method': [
        'Original',
        'Remove Invalid',
        'Cap (Winsorize)',
        'Log Transform',
        'Replace with Median',
        'Flag Only'
    ],
    'Rows': [
        len(df_outliers),
        len(df_removed),
        len(df_capped),
        len(df_transformed),
        len(df_replaced),
        len(df_flagged)
    ],
    'Age_Range': [
        f"{df_outliers['age'].min():.0f}-{df_outliers['age'].max():.0f}",
        f"{df_removed['age'].min():.0f}-{df_removed['age'].max():.0f}",
        f"{df_capped['age_capped'].min():.0f}-{df_capped['age_capped'].max():.0f}",
        f"{df_outliers['age'].min():.0f}-{df_outliers['age'].max():.0f}",
        f"{df_replaced['age'].min():.0f}-{df_replaced['age'].max():.0f}",
        f"{df_outliers['age'].min():.0f}-{df_outliers['age'].max():.0f}"
    ],
    'Description': [
        'No changes',
        'Removes rows with invalid values',
        'Caps extreme values at boundaries',
        'Log transformation to reduce skew',
        'Replaces outliers with median',
        'Keeps data, adds indicator columns'
    ]
})
print(summary.to_string(index=False))

## 7. Data Validation

**What is it?**
Data validation ensures that data meets specific business rules, constraints, and requirements before being used in analysis or modeling.

**Why use it?**
- Ensures data quality and reliability
- Prevents errors in downstream processes
- Maintains data integrity across systems
- Catches data entry errors early
- Ensures compliance with business rules

**When to use it?**
- After data cleaning operations
- Before loading data into databases
- When receiving data from external sources
- During ETL/ELT processes
- Before analysis or modeling

**Validation Types:**
1. **Type validation**: Ensure correct data types
2. **Range validation**: Values within acceptable ranges
3. **Format validation**: Data follows expected patterns
4. **Uniqueness validation**: Keys are unique
5. **Referential integrity**: Foreign keys exist in referenced tables
6. **Business rule validation**: Custom domain-specific rules

In [ ]:
# ============================================
# 7. DATA VALIDATION
# ============================================

df_validate = df.drop_duplicates(subset=['customer_id']).reset_index(drop=True).copy()

# Clean data first
df_validate['age'] = pd.to_numeric(df_validate['age'], errors='coerce')
df_validate['salary'] = df_validate['salary'].astype(str).str.replace(r'[\$,]', '', regex=True)
df_validate['salary'] = pd.to_numeric(df_validate['salary'], errors='coerce')
df_validate['email'] = df_validate['email'].str.strip().str.lower()

print("=" * 80)
print("DATA VALIDATION")
print("=" * 80)

# 1. Type validation
print("\n1. Type Validation:")

def validate_types(df):
    """Validate data types of columns"""
    expected_types = {
        'customer_id': ['int64', 'float64'],
        'name': ['object', 'string'],
        'email': ['object', 'string'],
        'age': ['int64', 'float64'],
        'salary': ['int64', 'float64'],
        'city': ['object', 'string'],
        'country': ['object', 'string'],
        'department': ['object', 'string']
    }
    
    issues = []
    for col, expected in expected_types.items():
        if col in df.columns:
            if df[col].dtype.name not in expected:
                issues.append(f"{col}: expected {expected}, got {df[col].dtype.name}")
    
    return issues

type_issues = validate_types(df_validate)
if type_issues:
    print("   Type validation issues:")
    for issue in type_issues:
        print(f"   - {issue}")
else:
    print("   ✓ All columns have correct types")

# 2. Range validation
print("\n2. Range Validation:")

def validate_ranges(df):
    """Validate that numeric values are within acceptable ranges"""
    rules = {
        'age': (18, 100),
        'salary': (10000, 500000),
        'customer_id': (1, 999999)
    }
    
    results = {}
    for col, (min_val, max_val) in rules.items():
        if col in df.columns:
            below_min = (df[col] < min_val).sum()
            above_max = (df[col] > max_val).sum()
            valid = len(df) - below_min - above_max - df[col].isna().sum()
            
            results[col] = {
                'valid': valid,
                'below_min': below_min,
                'above_max': above_max,
                'missing': df[col].isna().sum(),
                'range': f"{min_val}-{max_val}"
            }
    
    return results

range_results = validate_ranges(df_validate)
for col, stats in range_results.items():
    print(f"\n   {col} (range: {stats['range']}):")
    print(f"   Valid: {stats['valid']}, Below min: {stats['below_min']}, " 
          f"Above max: {stats['above_max']}, Missing: {stats['missing']}")

# 3. Format validation
print("\n3. Format Validation:")

def validate_email(email):
    """Validate email format"""
    pattern = r'^[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}$'
    if pd.isna(email):
        return False
    return bool(re.match(pattern, str(email)))

def validate_phone(phone):
    """Validate phone format (10 digits)"""
    if pd.isna(phone):
        return False
    digits = re.sub(r'\D', '', str(phone))
    return len(digits) == 10

# Validate emails
df_validate['email_valid'] = df_validate['email'].apply(validate_email)
invalid_emails = (~df_validate['email_valid']).sum()
print(f"   Email validation: {invalid_emails} invalid formats")
if invalid_emails > 0:
    print("   Invalid emails:")
    print(df_validate[~df_validate['email_valid']][['name', 'email']])

# Validate phone numbers
df_validate['phone_valid'] = df_validate['phone'].apply(validate_phone)
invalid_phones = (~df_validate['phone_valid']).sum()
print(f"\n   Phone validation: {invalid_phones} invalid formats")
if invalid_phones > 0:
    print("   Invalid phones:")
    print(df_validate[~df_validate['phone_valid']][['name', 'phone']])

# 4. Uniqueness validation
print("\n4. Uniqueness Validation:")

def check_uniqueness(df, columns):
    """Check if specified columns have unique values"""
    results = {}
    for col in columns:
        if col in df.columns:
            total = len(df)
            unique = df[col].nunique()
            duplicates = total - unique
            results[col] = {
                'total': total,
                'unique': unique,
                'duplicates': duplicates,
                'is_unique': duplicates == 0
            }
    return results

unique_cols = ['customer_id', 'email']
uniqueness_results = check_uniqueness(df_validate, unique_cols)

for col, stats in uniqueness_results.items():
    status = "✓ UNIQUE" if stats['is_unique'] else "✗ HAS DUPLICATES"
    print(f"   {col}: {status}")
    print(f"   Total: {stats['total']}, Unique: {stats['unique']}, Duplicates: {stats['duplicates']}")

# 5. Allowed values validation
print("\n5. Allowed Values Validation:")

allowed_values = {
    'department': ['sales', 'engineering', 'hr', 'marketing', 'finance'],
    'country': ['usa', 'uk', 'canada', 'australia']
}

def validate_allowed_values(df, rules):
    """Check if values are in allowed list"""
    results = {}
    for col, allowed in rules.items():
        if col in df.columns:
            # Normalize for comparison
            df_values = df[col].str.lower().str.strip()
            invalid = ~df_values.isin(allowed) & df_values.notna()
            invalid_count = invalid.sum()
            
            results[col] = {
                'invalid_count': invalid_count,
                'invalid_values': df[invalid][col].unique().tolist()
            }
    return results

allowed_results = validate_allowed_values(df_validate, allowed_values)

for col, stats in allowed_results.items():
    print(f"\n   {col}:")
    print(f"   Invalid count: {stats['invalid_count']}")
    if stats['invalid_values']:
        print(f"   Invalid values: {stats['invalid_values']}")

# 6. Completeness validation
print("\n6. Completeness Validation:")

def validate_completeness(df, required_columns):
    """Check that required columns have no missing values"""
    results = {}
    for col in required_columns:
        if col in df.columns:
            missing = df[col].isna().sum()
            completeness = (1 - missing / len(df)) * 100
            results[col] = {
                'missing': missing,
                'completeness': completeness,
                'is_complete': missing == 0
            }
    return results

required_cols = ['customer_id', 'name', 'email', 'department']
completeness_results = validate_completeness(df_validate, required_cols)

for col, stats in completeness_results.items():
    status = "✓ COMPLETE" if stats['is_complete'] else "✗ INCOMPLETE"
    print(f"   {col}: {status}")
    print(f"   Missing: {stats['missing']}, Completeness: {stats['completeness']:.1f}%")

# 7. Business rule validation
print("\n7. Business Rule Validation:")

def validate_business_rules(row):
    """Validate custom business rules"""
    errors = []
    
    # Rule 1: Sales department should have salary > 40000
    if pd.notna(row['department']) and pd.notna(row['salary']):
        if row['department'].lower() == 'sales' and row['salary'] < 40000:
            errors.append("Sales dept salary must be >= $40,000")
    
    # Rule 2: Engineering department should have salary > 60000
    if pd.notna(row['department']) and pd.notna(row['salary']):
        if row['department'].lower() == 'engineering' and row['salary'] < 60000:
            errors.append("Engineering dept salary must be >= $60,000")
    
    # Rule 3: Age and experience correlation (if we had experience field)
    # Young people (< 25) shouldn't have very high salaries (> 100000)
    if pd.notna(row['age']) and pd.notna(row['salary']):
        if row['age'] < 25 and row['salary'] > 100000:
            errors.append("Salary unusually high for age")
    
    return errors if errors else ['Valid']

df_validate['business_rule_errors'] = df_validate.apply(validate_business_rules, axis=1)
df_validate['business_rules_valid'] = df_validate['business_rule_errors'].apply(lambda x: x == ['Valid'])

violations = (~df_validate['business_rules_valid']).sum()
print(f"   Business rule violations: {violations}")
if violations > 0:
    print("\n   Records with violations:")
    print(df_validate[~df_validate['business_rules_valid']][['name', 'department', 'age', 'salary', 'business_rule_errors']])

# 8. Create comprehensive validation report
print("\n8. Comprehensive Validation Report:")

def create_validation_report(df):
    """Create a comprehensive validation report"""
    report = {
        'Total Records': len(df),
        'Type Issues': len(validate_types(df)),
        'Invalid Emails': (~df['email_valid']).sum() if 'email_valid' in df else 0,
        'Invalid Phones': (~df['phone_valid']).sum() if 'phone_valid' in df else 0,
        'Duplicate Customer IDs': len(df) - df['customer_id'].nunique(),
        'Missing Required Fields': sum([df[col].isna().sum() for col in ['customer_id', 'name', 'email'] if col in df.columns]),
        'Business Rule Violations': (~df['business_rules_valid']).sum() if 'business_rules_valid' in df else 0,
        'Overall Valid Records': 0
    }
    
    # Calculate overall valid records (no issues in any category)
    valid_mask = (
        df['email_valid'] & 
        df['phone_valid'] & 
        df['business_rules_valid']
    )
    report['Overall Valid Records'] = valid_mask.sum()
    report['Data Quality Score'] = f"{(valid_mask.sum() / len(df) * 100):.1f}%"
    
    return report

validation_report = create_validation_report(df_validate)

print("\n   VALIDATION SUMMARY:")
print("   " + "-" * 60)
for key, value in validation_report.items():
    print(f"   {key:.<40} {value:>15}")

# 9. Flag all validation issues
print("\n9. Creating Validation Flags:")

df_validate['has_type_issues'] = False  # Would be True if types don't match
df_validate['has_range_issues'] = (
    (df_validate['age'] < 18) | (df_validate['age'] > 100) |
    (df_validate['salary'] < 10000) | (df_validate['salary'] > 500000)
)
df_validate['has_format_issues'] = ~df_validate['email_valid'] | ~df_validate['phone_valid']
df_validate['has_business_issues'] = ~df_validate['business_rules_valid']

# Overall validation flag
df_validate['is_valid'] = (
    ~df_validate['has_type_issues'] &
    ~df_validate['has_range_issues'] &
    ~df_validate['has_format_issues'] &
    ~df_validate['has_business_issues']
)

print("   Validation status:")
print(df_validate['is_valid'].value_counts())

print("\n   Sample of invalid records:")
invalid_sample = df_validate[~df_validate['is_valid']].head()
print(invalid_sample[['name', 'email', 'has_range_issues', 'has_format_issues', 'has_business_issues']])

print("\n   ✓ Data validation complete!")

## 8. Dealing with Inconsistent Data

**What is it?**
Inconsistent data refers to data that doesn't follow standard formats, naming conventions, or representations across the dataset.

**Why use it?**
- Standardizes data for analysis
- Improves data quality and reliability
- Enables accurate grouping and aggregation
- Facilitates data integration from multiple sources
- Reduces errors in reporting and analytics

**When to use it?**
- When merging data from multiple sources
- Data has mixed formats or conventions
- Categorical variables have variations
- Units or scales differ across records
- Case sensitivity issues exist

**Common inconsistencies:**
1. **Case variations**: "USA" vs "usa" vs "Usa"
2. **Spelling variations**: "New York" vs "NY" vs "New York City"
3. **Unit differences**: "5000" vs "5K" vs "$5,000"
4. **Date formats**: "2024-01-15" vs "01/15/2024" vs "15-Jan-2024"
5. **Abbreviations**: "Street" vs "St." vs "St"
6. **Encoding issues**: Special characters, whitespace

In [ ]:
# ============================================
# 8. DEALING WITH INCONSISTENT DATA
# ============================================

# Create sample data with inconsistencies
inconsistent_data = {
    'product_id': [1, 2, 3, 4, 5, 6, 7, 8],
    'product_name': ['iPhone 14', 'iphone 14', 'IPHONE 14', 'iPhone14', 
                     'Samsung Galaxy', 'samsung galaxy', 'Galaxy Samsung', 'SamsungGalaxy'],
    'category': ['Electronics', 'electronics', 'ELECTRONICS', 'Electronics',
                'Electronics', 'Electronic', 'electronics', 'Electronics'],
    'price': ['$999', '999.00', '$999.00', '999',
             '$799', '799.00', '$799.00', '799'],
    'quantity': ['5', '5 units', '5pcs', '5 pieces',
                '10', '10 units', '10pcs', '10 pieces'],
    'weight': ['1.5kg', '1.5 kg', '1500g', '1.5',
              '2kg', '2.0 kg', '2000g', '2.0'],
    'country': ['USA', 'usa', 'U.S.A', 'United States',
               'UK', 'United Kingdom', 'U.K.', 'uk'],
    'status': ['Active', 'active', 'ACTIVE', 'Active',
              'Inactive', 'inactive', 'INACTIVE', 'In-Active']
}

df_inconsistent = pd.DataFrame(inconsistent_data)

print("=" * 80)
print("DEALING WITH INCONSISTENT DATA")
print("=" * 80)
print("\n1. Original Data with Inconsistencies:")
print(df_inconsistent)

# 2. Standardize case
print("\n2. Standardizing Case:")

df_clean = df_inconsistent.copy()

# Standardize text columns to title case
df_clean['product_name'] = df_clean['product_name'].str.lower().str.title()
df_clean['category'] = df_clean['category'].str.lower().str.title()
df_clean['status'] = df_clean['status'].str.lower().str.title()

print("   After case standardization:")
print(df_clean[['product_name', 'category', 'status']])

# 3. Remove whitespace inconsistencies
print("\n3. Removing Whitespace Inconsistencies:")

# Remove extra spaces
df_clean['product_name'] = df_clean['product_name'].str.replace(r'\s+', ' ', regex=True).str.strip()
df_clean['weight'] = df_clean['weight'].str.replace(r'\s+', '', regex=True)

print("   Product names after whitespace removal:")
print(df_clean['product_name'].unique())

# 4. Standardize product names
print("\n4. Standardizing Product Names:")

# Create mapping for product variations
product_mapping = {
    'iphone 14': 'iPhone 14',
    'iphone14': 'iPhone 14',
    'samsung galaxy': 'Samsung Galaxy',
    'galaxy samsung': 'Samsung Galaxy',
    'samsunggalaxy': 'Samsung Galaxy'
}

# Apply standardization (case-insensitive)
df_clean['product_name_std'] = df_clean['product_name'].str.lower().map(product_mapping)
df_clean['product_name_std'] = df_clean['product_name_std'].fillna(df_clean['product_name'])

print("   Original → Standardized:")
for orig, std in zip(df_inconsistent['product_name'], df_clean['product_name_std']):
    print(f"   {orig:20} → {std}")

# 5. Standardize price format
print("\n5. Standardizing Price Format:")

def standardize_price(price):
    """Convert various price formats to float"""
    # Remove currency symbols and spaces
    price_str = str(price).replace('$', '').replace(',', '').strip()
    try:
        return float(price_str)
    except:
        return None

df_clean['price_std'] = df_clean['price'].apply(standardize_price)

print("   Original → Standardized:")
for orig, std in zip(df_inconsistent['price'], df_clean['price_std']):
    print(f"   {orig:10} → ${std:.2f}")

# 6. Standardize quantity format
print("\n6. Standardizing Quantity Format:")

def extract_quantity(qty):
    """Extract numeric quantity from various formats"""
    # Extract digits only
    numbers = re.findall(r'\d+', str(qty))
    return int(numbers[0]) if numbers else None

df_clean['quantity_std'] = df_clean['quantity'].apply(extract_quantity)

print("   Original → Standardized:")
for orig, std in zip(df_inconsistent['quantity'], df_clean['quantity_std']):
    print(f"   {orig:15} → {std}")

# 7. Standardize weight to kg
print("\n7. Standardizing Weight (convert to kg):")

def standardize_weight(weight):
    """Convert various weight formats to kg"""
    weight_str = str(weight).lower().strip()
    
    # Extract number
    numbers = re.findall(r'\d+\.?\d*', weight_str)
    if not numbers:
        return None
    
    value = float(numbers[0])
    
    # Convert based on unit
    if 'g' in weight_str and 'kg' not in weight_str:
        value = value / 1000  # grams to kg
    elif 'lb' in weight_str:
        value = value * 0.453592  # pounds to kg
    
    return round(value, 2)

df_clean['weight_kg'] = df_clean['weight'].apply(standardize_weight)

print("   Original → Standardized (kg):")
for orig, std in zip(df_inconsistent['weight'], df_clean['weight_kg']):
    print(f"   {orig:10} → {std} kg")

# 8. Standardize country names
print("\n8. Standardizing Country Names:")

# Create comprehensive country mapping
country_mapping = {
    'usa': 'United States',
    'u.s.a': 'United States',
    'u.s.': 'United States',
    'united states': 'United States',
    'us': 'United States',
    'uk': 'United Kingdom',
    'u.k.': 'United Kingdom',
    'united kingdom': 'United Kingdom',
    'britain': 'United Kingdom',
    'great britain': 'United Kingdom'
}

df_clean['country_std'] = df_clean['country'].str.lower().map(country_mapping)
df_clean['country_std'] = df_clean['country_std'].fillna(df_clean['country'])

print("   Original → Standardized:")
for orig, std in zip(df_inconsistent['country'], df_clean['country_std']):
    print(f"   {orig:20} → {std}")

# 9. Standardize status values
print("\n9. Standardizing Status Values:")

# Normalize status
df_clean['status_std'] = df_clean['status'].str.lower().str.replace('-', '').str.replace('_', '')

# Map to standard values
status_mapping = {
    'active': 'Active',
    'inactive': 'Inactive'
}

df_clean['status_std'] = df_clean['status_std'].map(status_mapping)

print("   Original → Standardized:")
for orig, std in zip(df_inconsistent['status'], df_clean['status_std']):
    print(f"   {orig:15} → {std}")

# 10. Standardize category names
print("\n10. Standardizing Category Names:")

# Fix common misspellings and variations
category_mapping = {
    'electronics': 'Electronics',
    'electronic': 'Electronics',  # Singular to plural
    'electronix': 'Electronics',  # Common misspelling
}

df_clean['category_std'] = df_clean['category'].str.lower().map(category_mapping)
df_clean['category_std'] = df_clean['category_std'].fillna(df_clean['category'])

print("   Unique categories before standardization:", df_inconsistent['category'].nunique())
print("   Unique categories after standardization:", df_clean['category_std'].nunique())
print("   Standardized categories:", df_clean['category_std'].unique())

# 11. Create final cleaned dataset
print("\n11. Final Cleaned Dataset:")

df_final = df_clean[[
    'product_id',
    'product_name_std',
    'category_std',
    'price_std',
    'quantity_std',
    'weight_kg',
    'country_std',
    'status_std'
]].rename(columns={
    'product_name_std': 'product_name',
    'category_std': 'category',
    'price_std': 'price',
    'quantity_std': 'quantity',
    'weight_kg': 'weight_kg',
    'country_std': 'country',
    'status_std': 'status'
})

print(df_final)

# 12. Compare before and after
print("\n12. Comparison: Before vs After")

comparison = pd.DataFrame({
    'Metric': [
        'Unique Product Names',
        'Unique Categories',
        'Unique Countries',
        'Unique Status Values',
        'Price Format',
        'Quantity Format',
        'Weight Format'
    ],
    'Before': [
        df_inconsistent['product_name'].nunique(),
        df_inconsistent['category'].nunique(),
        df_inconsistent['country'].nunique(),
        df_inconsistent['status'].nunique(),
        'Mixed',
        'Mixed',
        'Mixed'
    ],
    'After': [
        df_final['product_name'].nunique(),
        df_final['category'].nunique(),
        df_final['country'].nunique(),
        df_final['status'].nunique(),
        'Numeric (USD)',
        'Integer',
        'Float (kg)'
    ]
})

print(comparison.to_string(index=False))

# 13. Create a standardization function
print("\n13. Reusable Standardization Function:")

def standardize_dataset(df, mappings):
    """
    Generic function to standardize a dataset
    
    Parameters:
    - df: DataFrame to standardize
    - mappings: dict with column names as keys and standardization rules as values
    """
    df_std = df.copy()
    
    for column, rules in mappings.items():
        if column not in df_std.columns:
            continue
            
        rule_type = rules.get('type', 'mapping')
        
        if rule_type == 'mapping':
            # Use provided mapping dictionary
            mapping = rules.get('mapping', {})
            df_std[column] = df_std[column].str.lower().map(mapping)
            df_std[column] = df_std[column].fillna(df[column])
            
        elif rule_type == 'case':
            # Standardize case
            case_type = rules.get('case', 'lower')
            if case_type == 'lower':
                df_std[column] = df_std[column].str.lower()
            elif case_type == 'upper':
                df_std[column] = df_std[column].str.upper()
            elif case_type == 'title':
                df_std[column] = df_std[column].str.title()
                
        elif rule_type == 'numeric':
            # Extract numeric values
            df_std[column] = pd.to_numeric(
                df_std[column].astype(str).str.extract(r'(\d+\.?\d*)', expand=False),
                errors='coerce'
            )
    
    return df_std

# Example usage
standardization_rules = {
    'country': {
        'type': 'mapping',
        'mapping': country_mapping
    },
    'status': {
        'type': 'case',
        'case': 'title'
    },
    'price': {
        'type': 'numeric'
    }
}

print("   ✓ Standardization function created")
print("   Usage: standardize_dataset(df, standardization_rules)")

print("\n   ✓ Data inconsistency handling complete!")

## 9. Date & Time Data Cleaning

**What is it?**
Date and time cleaning involves parsing, validating, and standardizing temporal data into consistent formats.

**Why use it?**
- Enables time-based analysis and calculations
- Ensures consistency across time zones
- Handles various date formats
- Extracts meaningful temporal features
- Fixes invalid or ambiguous dates

**When to use it?**
- Data has mixed date formats
- Time zone conversions needed
- Invalid dates present
- Need to extract temporal features (day, month, year)
- Handling timestamps from different systems
- Time series analysis

**Common date/time issues:**
1. **Format variations**: "2024-01-15", "01/15/2024", "15-Jan-2024"
2. **Time zones**: UTC, EST, PST conversions
3. **Invalid dates**: "2024-02-30", "13/45/2024"
4. **Missing time components**: Dates without times
5. **Ambiguous formats**: "01/02/2024" (Jan 2 or Feb 1?)
6. **Epoch timestamps**: Unix timestamps vs datetime objects

In [ ]:
# ============================================
# 9. DATE & TIME DATA CLEANING
# ============================================

from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

# Create sample data with date/time issues
datetime_data = {
    'transaction_id': range(1, 13),
    'date_str': [
        '2024-01-15',           # ISO format
        '01/15/2024',           # US format
        '15-Jan-2024',          # Day-Month-Year with month name
        '2024/01/15',           # ISO with slashes
        '15.01.2024',           # European format
        '2024-02-30',           # Invalid date
        'Jan 15, 2024',         # Month Day, Year
        '2024-01-15 14:30:00',  # With time
        '15/01/2024',           # Ambiguous (could be Jan 15 or May 1)
        '2024-W03-1',           # Week format
        '',                     # Missing
        '1705334400'            # Unix timestamp
    ],
    'time_str': [
        '14:30:00',
        '2:30 PM',
        '14:30',
        '02:30:00 PM',
        '143000',
        '25:70:00',  # Invalid time
        '14.30.00',
        '14:30:00.000',
        '2:30p',
        '14h30m',
        '',
        '1705334400'
    ],
    'timezone': [
        'UTC', 'EST', 'PST', 'UTC', 'CET',
        'UTC', 'EST', 'PST', 'UTC', 'EST',
        'UTC', 'UTC'
    ]
}

df_datetime = pd.DataFrame(datetime_data)

print("=" * 80)
print("DATE & TIME DATA CLEANING")
print("=" * 80)
print("\n1. Original Data with Various Date/Time Formats:")
print(df_datetime)

# 2. Parse multiple date formats
print("\n2. Parsing Multiple Date Formats:")

def parse_date_flexible(date_str):
    """Try multiple date formats to parse a date string"""
    if pd.isna(date_str) or date_str == '':
        return pd.NaT
    
    date_str = str(date_str).strip()
    
    # Try different formats
    formats = [
        '%Y-%m-%d',           # 2024-01-15
        '%m/%d/%Y',           # 01/15/2024
        '%d-%b-%Y',           # 15-Jan-2024
        '%Y/%m/%d',           # 2024/01/15
        '%d.%m.%Y',           # 15.01.2024
        '%b %d, %Y',          # Jan 15, 2024
        '%Y-%m-%d %H:%M:%S',  # 2024-01-15 14:30:00
        '%d/%m/%Y',           # 15/01/2024 (European)
    ]
    
    # Try Unix timestamp
    try:
        timestamp = float(date_str)
        if timestamp > 1000000000:  # Likely a Unix timestamp
            return pd.to_datetime(timestamp, unit='s')
    except:
        pass
    
    # Try each format
    for fmt in formats:
        try:
            return pd.to_datetime(date_str, format=fmt)
        except:
            continue
    
    # If all fail, return NaT
    return pd.NaT

df_datetime['date_parsed'] = df_datetime['date_str'].apply(parse_date_flexible)

print("   Original → Parsed:")
for orig, parsed in zip(df_datetime['date_str'], df_datetime['date_parsed']):
    status = "✓" if pd.notna(parsed) else "✗"
    print(f"   {status} {str(orig):25} → {parsed}")

# 3. Identify invalid dates
print("\n3. Identifying Invalid Dates:")

invalid_dates = df_datetime['date_parsed'].isna()
print(f"   Invalid/unparseable dates: {invalid_dates.sum()}")
if invalid_dates.sum() > 0:
    print("   Invalid date strings:")
    print(df_datetime[invalid_dates][['transaction_id', 'date_str']])

# 4. Parse time formats
print("\n4. Parsing Time Formats:")

def parse_time_flexible(time_str):
    """Parse various time formats"""
    if pd.isna(time_str) or time_str == '':
        return pd.NaT
    
    time_str = str(time_str).strip()
    
    # Try Unix timestamp
    try:
        timestamp = float(time_str)
        if timestamp > 1000000000:
            dt = pd.to_datetime(timestamp, unit='s')
            return dt.time()
    except:
        pass
    
    # Handle formats like '143000' (HHMMSS)
    if time_str.isdigit() and len(time_str) == 6:
        try:
            return datetime.strptime(time_str, '%H%M%S').time()
        except:
            pass
    
    # Handle formats like '14h30m'
    if 'h' in time_str.lower() and 'm' in time_str.lower():
        try:
            parts = time_str.lower().replace('h', ':').replace('m', '').split(':')
            return datetime.strptime(f"{parts[0]}:{parts[1]}:00", '%H:%M:%S').time()
        except:
            pass
    
    # Standard formats
    formats = [
        '%H:%M:%S',           # 14:30:00
        '%I:%M %p',           # 2:30 PM
        '%H:%M',              # 14:30
        '%I:%M:%S %p',        # 02:30:00 PM
        '%H.%M.%S',           # 14.30.00
        '%H:%M:%S.%f',        # 14:30:00.000
        '%I:%M%p',            # 2:30PM
    ]
    
    for fmt in formats:
        try:
            return datetime.strptime(time_str, fmt).time()
        except:
            continue
    
    return pd.NaT

df_datetime['time_parsed'] = df_datetime['time_str'].apply(parse_time_flexible)

print("   Original → Parsed:")
for orig, parsed in zip(df_datetime['time_str'], df_datetime['time_parsed']):
    status = "✓" if pd.notna(parsed) else "✗"
    print(f"   {status} {str(orig):20} → {parsed}")

# 5. Combine date and time
print("\n5. Combining Date and Time:")

def combine_datetime(date, time):
    """Combine date and time into datetime"""
    if pd.isna(date):
        return pd.NaT
    
    if pd.isna(time):
        # Return date with midnight time
        return pd.to_datetime(date)
    
    # Combine date and time
    return pd.to_datetime(f"{date.date()} {time}")

df_datetime['datetime_combined'] = df_datetime.apply(
    lambda row: combine_datetime(row['date_parsed'], row['time_parsed']),
    axis=1
)

print("   Combined datetime:")
print(df_datetime[['date_parsed', 'time_parsed', 'datetime_combined']].head(10))

# 6. Extract date components
print("\n6. Extracting Date Components:")

df_datetime['year'] = df_datetime['date_parsed'].dt.year
df_datetime['month'] = df_datetime['date_parsed'].dt.month
df_datetime['day'] = df_datetime['date_parsed'].dt.day
df_datetime['day_of_week'] = df_datetime['date_parsed'].dt.day_name()
df_datetime['week_of_year'] = df_datetime['date_parsed'].dt.isocalendar().week
df_datetime['quarter'] = df_datetime['date_parsed'].dt.quarter
df_datetime['is_weekend'] = df_datetime['date_parsed'].dt.dayofweek >= 5

print("   Date components extracted:")
print(df_datetime[['date_parsed', 'year', 'month', 'day', 'day_of_week', 'is_weekend']].head(8))

# 7. Handle time zones
print("\n7. Handling Time Zones:")

# Create timezone-aware datetime (for valid dates)
df_datetime['datetime_utc'] = pd.to_datetime(df_datetime['datetime_combined'], utc=True, errors='coerce')

print("   UTC datetime:")
print(df_datetime[['datetime_combined', 'timezone', 'datetime_utc']].head(8))

# 8. Calculate time differences
print("\n8. Calculating Time Differences:")

# Create a reference date
reference_date = pd.to_datetime('2024-01-15')

df_datetime['days_from_reference'] = (df_datetime['date_parsed'] - reference_date).dt.days

print("   Days from reference date (2024-01-15):")
print(df_datetime[['date_parsed', 'days_from_reference']].head(8))

# 9. Validate date ranges
print("\n9. Validating Date Ranges:")

def validate_date_range(date, min_date='2020-01-01', max_date='2030-12-31'):
    """Check if date is within valid range"""
    if pd.isna(date):
        return False
    
    min_dt = pd.to_datetime(min_date)
    max_dt = pd.to_datetime(max_date)
    
    return min_dt <= date <= max_dt

df_datetime['date_valid'] = df_datetime['date_parsed'].apply(validate_date_range)

print(f"   Valid dates (2020-2030): {df_datetime['date_valid'].sum()}")
print(f"   Invalid/out-of-range dates: {(~df_datetime['date_valid']).sum()}")

# 10. Handle missing dates
print("\n10. Handling Missing Dates:")

df_datetime_filled = df_datetime.copy()

# Strategy 1: Fill with reference date
df_datetime_filled['date_filled_ref'] = df_datetime_filled['date_parsed'].fillna(reference_date)

# Strategy 2: Forward fill
df_datetime_filled['date_filled_ffill'] = df_datetime_filled['date_parsed'].fillna(method='ffill')

# Strategy 3: Backward fill
df_datetime_filled['date_filled_bfill'] = df_datetime_filled['date_parsed'].fillna(method='bfill')

print("   Missing date handling strategies:")
print(df_datetime_filled[['date_str', 'date_parsed', 'date_filled_ref', 'date_filled_ffill']].tail(5))

# 11. Standardize to ISO format
print("\n11. Standardizing to ISO Format:")

df_datetime['date_iso'] = df_datetime['date_parsed'].dt.strftime('%Y-%m-%d')
df_datetime['datetime_iso'] = df_datetime['datetime_combined'].dt.strftime('%Y-%m-%d %H:%M:%S')

print("   Standardized formats:")
print(df_datetime[['date_str', 'date_iso', 'datetime_iso']].head(8))

# 12. Create date bins
print("\n12. Creating Date Categories/Bins:")

# Categorize by month
df_datetime['month_name'] = df_datetime['date_parsed'].dt.strftime('%B')

# Categorize by season (Northern Hemisphere)
def get_season(month):
    if pd.isna(month):
        return 'Unknown'
    if month in [12, 1, 2]:
        return 'Winter'
    elif month in [3, 4, 5]:
        return 'Spring'
    elif month in [6, 7, 8]:
        return 'Summer'
    else:
        return 'Fall'

df_datetime['season'] = df_datetime['month'].apply(get_season)

# Categorize by time of day
def get_time_of_day(time):
    if pd.isna(time):
        return 'Unknown'
    hour = time.hour if hasattr(time, 'hour') else 0
    if 5 <= hour < 12:
        return 'Morning'
    elif 12 <= hour < 17:
        return 'Afternoon'
    elif 17 <= hour < 21:
        return 'Evening'
    else:
        return 'Night'

df_datetime['time_of_day'] = df_datetime['time_parsed'].apply(get_time_of_day)

print("   Temporal categories:")
print(df_datetime[['date_parsed', 'month_name', 'season', 'time_of_day']].head(8))

# 13. Summary
print("\n13. Date/Time Cleaning Summary:")

summary = pd.DataFrame({
    'Metric': [
        'Total Records',
        'Successfully Parsed Dates',
        'Failed to Parse Dates',
        'Successfully Parsed Times',
        'Failed to Parse Times',
        'Valid Dates (in range)',
        'Invalid Dates (out of range)',
        'Dates with Timezone Info'
    ],
    'Count': [
        len(df_datetime),
        df_datetime['date_parsed'].notna().sum(),
        df_datetime['date_parsed'].isna().sum(),
        df_datetime['time_parsed'].notna().sum(),
        df_datetime['time_parsed'].isna().sum(),
        df_datetime['date_valid'].sum(),
        (~df_datetime['date_valid']).sum(),
        df_datetime['datetime_utc'].notna().sum()
    ]
})

print(summary.to_string(index=False))

print("\n   ✓ Date & time cleaning complete!")

## 10. Categorical Data Cleaning

**What is it?**
Categorical data cleaning involves standardizing, encoding, and optimizing categorical variables for analysis and machine learning.

**Why use it?**
- Reduces cardinality (number of unique values)
- Standardizes category names
- Handles misspellings and variations
- Prepares data for machine learning models
- Improves memory efficiency
- Enables proper grouping and analysis

**When to use it?**
- Categories have typos or variations
- Too many unique categories (high cardinality)
- Need to encode for machine learning
- Categories need hierarchical grouping
- Memory optimization needed
- Rare categories need grouping

**Common categorical operations:**
1. **Standardization**: Fix typos, case, spacing
2. **Encoding**: Convert to numeric (label, one-hot, target)
3. **Grouping**: Combine rare/similar categories
4. **Validation**: Ensure only allowed values
5. **Ordering**: Create ordinal categories

In [ ]:
# ============================================
# 10. CATEGORICAL DATA CLEANING
# ============================================

# Create sample categorical data with issues
categorical_data = {
    'customer_id': range(1, 21),
    'education': [
        'High School', 'high school', 'HS', 'High Scool',
        'Bachelor', 'bachelors', "Bachelor's", 'BS',
        'Master', 'masters', "Master's", 'MS', 'MBA',
        'PhD', 'phd', 'Doctorate',
        'Associate', 'associates', 'AA',
        'Some College'
    ],
    'income_level': [
        'Low', 'low', 'LOW', 'Medium',
        'medium', 'MEDIUM', 'High', 'high',
        'HIGH', 'Very High', 'very high',
        'Low', 'Medium', 'High',
        'low', 'medium', 'high',
        'Very Low', 'very low', 'Medium'
    ],
    'job_title': [
        'Software Engineer', 'software engineer', 'SWE',
        'Data Scientist', 'data scientist', 'DS',
        'Product Manager', 'product manager', 'PM',
        'Marketing Manager', 'marketing mgr', 'Mktg Mgr',
        'Sales Rep', 'sales representative', 'Sales',
        'CEO', 'Chief Executive Officer', 'ceo',
        'Analyst', 'Data Analyst'
    ],
    'city': [
        'New York', 'NYC', 'new york', 'New York City',
        'Los Angeles', 'LA', 'los angeles',
        'San Francisco', 'SF', 'san francisco',
        'Chicago', 'chicago',
        'Boston', 'boston',
        'Seattle', 'seattle',
        'Austin', 'austin',
        'Denver', 'denver'
    ]
}

df_categorical = pd.DataFrame(categorical_data)

print("=" * 80)
print("CATEGORICAL DATA CLEANING")
print("=" * 80)
print("\n1. Original Categorical Data:")
print(df_categorical)

print("\n   Initial Category Counts:")
for col in ['education', 'income_level', 'job_title', 'city']:
    print(f"   {col}: {df_categorical[col].nunique()} unique values")

# 2. Standardize case and spacing
print("\n2. Standardizing Case and Spacing:")

df_clean = df_categorical.copy()

# Standardize to title case
for col in ['education', 'income_level', 'job_title', 'city']:
    df_clean[f'{col}_std'] = df_clean[col].str.strip().str.title()

print("   After case standardization:")
for col in ['education', 'income_level', 'job_title', 'city']:
    print(f"   {col}: {df_clean[f'{col}_std'].nunique()} unique values")

# 3. Fix typos and variations using mapping
print("\n3. Fixing Typos and Variations:")

# Education mapping
education_mapping = {
    'high school': 'High School',
    'hs': 'High School',
    'high scool': 'High School',  # typo
    'bachelor': 'Bachelor\'s Degree',
    'bachelors': 'Bachelor\'s Degree',
    "bachelor's": 'Bachelor\'s Degree',
    'bs': 'Bachelor\'s Degree',
    'master': 'Master\'s Degree',
    'masters': 'Master\'s Degree',
    "master's": 'Master\'s Degree',
    'ms': 'Master\'s Degree',
    'mba': 'Master\'s Degree',
    'phd': 'Doctorate',
    'doctorate': 'Doctorate',
    'associate': 'Associate Degree',
    'associates': 'Associate Degree',
    'aa': 'Associate Degree',
    'some college': 'Some College'
}

df_clean['education_clean'] = df_clean['education'].str.lower().map(education_mapping)
df_clean['education_clean'] = df_clean['education_clean'].fillna(df_clean['education'])

print(f"   Education: {df_categorical['education'].nunique()} → {df_clean['education_clean'].nunique()} categories")
print(f"   Unique values: {sorted(df_clean['education_clean'].unique())}")

# Income level mapping
income_mapping = {
    'low': 'Low',
    'very low': 'Low',
    'medium': 'Medium',
    'high': 'High',
    'very high': 'High'
}

df_clean['income_clean'] = df_clean['income_level'].str.lower().map(income_mapping)

print(f"\n   Income Level: {df_categorical['income_level'].nunique()} → {df_clean['income_clean'].nunique()} categories")
print(f"   Unique values: {sorted(df_clean['income_clean'].unique())}")

# 4. Standardize abbreviations
print("\n4. Standardizing Abbreviations:")

# Job title mapping
job_mapping = {
    'software engineer': 'Software Engineer',
    'swe': 'Software Engineer',
    'data scientist': 'Data Scientist',
    'ds': 'Data Scientist',
    'product manager': 'Product Manager',
    'pm': 'Product Manager',
    'marketing manager': 'Marketing Manager',
    'marketing mgr': 'Marketing Manager',
    'mktg mgr': 'Marketing Manager',
    'sales rep': 'Sales Representative',
    'sales representative': 'Sales Representative',
    'sales': 'Sales Representative',
    'ceo': 'Chief Executive Officer',
    'chief executive officer': 'Chief Executive Officer',
    'analyst': 'Analyst',
    'data analyst': 'Data Analyst'
}

df_clean['job_title_clean'] = df_clean['job_title'].str.lower().map(job_mapping)
df_clean['job_title_clean'] = df_clean['job_title_clean'].fillna(df_clean['job_title'])

print(f"   Job Title: {df_categorical['job_title'].nunique()} → {df_clean['job_title_clean'].nunique()} categories")

# City mapping
city_mapping = {
    'new york': 'New York',
    'nyc': 'New York',
    'new york city': 'New York',
    'los angeles': 'Los Angeles',
    'la': 'Los Angeles',
    'san francisco': 'San Francisco',
    'sf': 'San Francisco',
    'chicago': 'Chicago',
    'boston': 'Boston',
    'seattle': 'Seattle',
    'austin': 'Austin',
    'denver': 'Denver'
}

df_clean['city_clean'] = df_clean['city'].str.lower().map(city_mapping)

print(f"   City: {df_categorical['city'].nunique()} → {df_clean['city_clean'].nunique()} categories")

# 5. Create hierarchical categories
print("\n5. Creating Hierarchical Categories:")

# Group education into broader categories
def categorize_education(education):
    if education in ['High School', 'Some College']:
        return 'Undergraduate Level'
    elif education == 'Associate Degree':
        return 'Associate Level'
    elif education == 'Bachelor\'s Degree':
        return 'Bachelor Level'
    elif education == 'Master\'s Degree':
        return 'Graduate Level'
    elif education == 'Doctorate':
        return 'Doctoral Level'
    else:
        return 'Other'

df_clean['education_level'] = df_clean['education_clean'].apply(categorize_education)

print("   Education hierarchy:")
print(df_clean.groupby(['education_level', 'education_clean']).size())

# 6. Handle rare categories
print("\n6. Handling Rare Categories:")

# Identify rare job titles (appearing less than 3 times)
value_counts = df_clean['job_title_clean'].value_counts()
rare_threshold = 3

rare_jobs = value_counts[value_counts < rare_threshold].index.tolist()
print(f"   Rare job titles (< {rare_threshold} occurrences): {rare_jobs}")

# Group rare categories
df_clean['job_title_grouped'] = df_clean['job_title_clean'].apply(
    lambda x: 'Other' if x in rare_jobs else x
)

print(f"\n   Job titles before grouping: {df_clean['job_title_clean'].nunique()}")
print(f"   Job titles after grouping: {df_clean['job_title_grouped'].nunique()}")

# 7. Label Encoding
print("\n7. Label Encoding (Ordinal):")

# Create ordinal encoding for education
education_order = [
    'High School',
    'Some College',
    'Associate Degree',
    'Bachelor\'s Degree',
    'Master\'s Degree',
    'Doctorate'
]

df_clean['education_encoded'] = pd.Categorical(
    df_clean['education_clean'],
    categories=education_order,
    ordered=True
).codes

print("   Education Label Encoding:")
encoding_map = df_clean[['education_clean', 'education_encoded']].drop_duplicates().sort_values('education_encoded')
print(encoding_map)

# Income level encoding
income_order = ['Low', 'Medium', 'High']
df_clean['income_encoded'] = pd.Categorical(
    df_clean['income_clean'],
    categories=income_order,
    ordered=True
).codes

print("\n   Income Level Label Encoding:")
print(df_clean[['income_clean', 'income_encoded']].drop_duplicates().sort_values('income_encoded'))

# 8. One-Hot Encoding
print("\n8. One-Hot Encoding (Nominal):")

# One-hot encode city
city_dummies = pd.get_dummies(df_clean['city_clean'], prefix='city')
print(f"   Created {len(city_dummies.columns)} binary columns for cities:")
print(f"   {list(city_dummies.columns)}")

print("\n   Sample of one-hot encoded data:")
print(city_dummies.head())

# 9. Frequency Encoding
print("\n9. Frequency Encoding:")

# Encode based on frequency
job_freq = df_clean['job_title_grouped'].value_counts(normalize=True)
df_clean['job_frequency'] = df_clean['job_title_grouped'].map(job_freq)

print("   Job Title Frequency Encoding:")
print(df_clean[['job_title_grouped', 'job_frequency']].drop_duplicates().sort_values('job_frequency', ascending=False))

# 10. Convert to category dtype for memory optimization
print("\n10. Memory Optimization with Category Dtype:")

# Check memory before
memory_before = df_clean[['education_clean', 'income_clean', 'job_title_clean', 'city_clean']].memory_usage(deep=True).sum()

# Convert to category
for col in ['education_clean', 'income_clean', 'job_title_grouped', 'city_clean']:
    df_clean[f'{col}_cat'] = df_clean[col].astype('category')

# Check memory after
memory_after = df_clean[['education_clean_cat', 'income_clean_cat', 'job_title_grouped_cat', 'city_clean_cat']].memory_usage(deep=True).sum()

print(f"   Memory before: {memory_before:,} bytes")
print(f"   Memory after: {memory_after:,} bytes")
print(f"   Savings: {memory_before - memory_after:,} bytes ({(1 - memory_after/memory_before)*100:.1f}% reduction)")

# 11. Validate categories
print("\n11. Validating Categories:")

def validate_categories(df, column, allowed_values):
    """Check if all values are in allowed list"""
    invalid = ~df[column].isin(allowed_values)
    invalid_count = invalid.sum()
    
    return {
        'column': column,
        'total': len(df),
        'valid': len(df) - invalid_count,
        'invalid': invalid_count,
        'invalid_values': df[invalid][column].unique().tolist() if invalid_count > 0 else []
    }

allowed_education = list(education_order)
validation = validate_categories(df_clean, 'education_clean', allowed_education)

print(f"   Education validation:")
print(f"   Valid: {validation['valid']}/{validation['total']}")
if validation['invalid'] > 0:
    print(f"   Invalid values: {validation['invalid_values']}")

# 12. Summary
print("\n12. Categorical Cleaning Summary:")

summary = pd.DataFrame({
    'Column': ['Education', 'Income Level', 'Job Title', 'City'],
    'Original_Unique': [
        df_categorical['education'].nunique(),
        df_categorical['income_level'].nunique(),
        df_categorical['job_title'].nunique(),
        df_categorical['city'].nunique()
    ],
    'Cleaned_Unique': [
        df_clean['education_clean'].nunique(),
        df_clean['income_clean'].nunique(),
        df_clean['job_title_clean'].nunique(),
        df_clean['city_clean'].nunique()
    ],
    'Grouped_Unique': [
        df_clean['education_level'].nunique(),
        df_clean['income_clean'].nunique(),
        df_clean['job_title_grouped'].nunique(),
        df_clean['city_clean'].nunique()
    ],
    'Encoding': [
        'Label (Ordinal)',
        'Label (Ordinal)',
        'Frequency',
        'One-Hot'
    ]
})

print(summary.to_string(index=False))

print("\n   ✓ Categorical data cleaning complete!")

## 11. Numerical Data Cleaning

**What is it?**
Numerical data cleaning involves handling errors, inconsistencies, and anomalies in numeric columns.

**Why use it?**
- Ensures accurate calculations
- Handles precision and rounding issues
- Converts units consistently
- Removes impossible values
- Prepares data for statistical analysis
- Handles special numeric values (infinity, NaN)

**When to use it?**
- Mixed numeric formats present
- Unit conversions needed
- Precision issues exist
- Negative values where impossible
- Need to bin continuous variables
- Scale differences across features

**Common numerical operations:**
1. **Type conversion**: String to numeric, handle errors
2. **Unit standardization**: Convert to common units
3. **Rounding**: Control decimal precision
4. **Binning**: Discretize continuous variables
5. **Validation**: Check ranges and constraints
6. **Special values**: Handle infinity, NaN, zeros

In [ ]:
# ============================================
# 11. NUMERICAL DATA CLEANING
# ============================================

# Create sample numerical data with issues
numerical_data = {
    'product_id': range(1, 16),
    'price': [
        '19.99', '29.99', 'N/A', '39.99',
        '49.99', 'inf', '59.99', '69.99',
        '-10.50', '79.99', '89.99', '99.99',
        '109.9999', '119.99', '0'
    ],
    'quantity': [
        100, 200, 150, 175,
        -50, 225, 250, 275,
        300, 999999, 350, 375,
        400, 425, 0
    ],
    'weight': [
        '1.5kg', '2.3 kg', '500g', '1200g',
        '3.5 kg', '2kg', '800g', '1.8kg',
        '2.5 kg', '3000g', '1.2 kg', '900g',
        '2.2kg', '1.7 kg', '0.5kg'
    ],
    'rating': [
        4.5, 4.7, 3.8, 4.2,
        4.9, 5.5, 4.1, 3.9,
        4.6, 0, 4.3, 4.8,
        -1.5, 4.4, np.inf
    ],
    'discount_pct': [
        '10%', '15%', '20%', '5%',
        '0%', '25%', '30%', '12%',
        '8%', '150%', '18%', '22%',
        '-5%', '50%', '100%'
    ]
}

df_numerical = pd.DataFrame(numerical_data)

print("=" * 80)
print("NUMERICAL DATA CLEANING")
print("=" * 80)
print("\n1. Original Numerical Data:")
print(df_numerical)
print("\n   Data types:")
print(df_numerical.dtypes)

# 2. Convert strings to numeric
print("\n2. Converting Strings to Numeric:")

df_clean = df_numerical.copy()

# Convert price (handle 'N/A', 'inf', etc.)
df_clean['price_numeric'] = pd.to_numeric(df_clean['price'], errors='coerce')

print("   Price conversion:")
for orig, converted in zip(df_numerical['price'], df_clean['price_numeric']):
    status = "✓" if pd.notna(converted) and np.isfinite(converted) else "✗"
    print(f"   {status} {str(orig):10} → {converted}")

# Convert discount percentage
df_clean['discount_numeric'] = df_clean['discount_pct'].str.replace('%', '').astype(float)

print("\n   Discount conversion:")
print(df_clean[['discount_pct', 'discount_numeric']].head(8))

# 3. Handle special numeric values
print("\n3. Handling Special Numeric Values:")

# Check for infinity
has_inf = np.isinf(df_clean['price_numeric']).sum()
has_ninf = np.isneginf(df_clean['price_numeric']).sum()
has_nan = df_clean['price_numeric'].isna().sum()

print(f"   Price column:")
print(f"   - Infinity values: {has_inf}")
print(f"   - Negative infinity: {has_ninf}")
print(f"   - NaN values: {has_nan}")

# Replace infinity with NaN
df_clean['price_clean'] = df_clean['price_numeric'].replace([np.inf, -np.inf], np.nan)

# Replace NaN with median
price_median = df_clean['price_clean'].median()
df_clean['price_clean'] = df_clean['price_clean'].fillna(price_median)

print(f"   Replaced inf/NaN with median: ${price_median:.2f}")

# 4. Validate numeric ranges
print("\n4. Validating Numeric Ranges:")

# Price should be positive and < 500
invalid_price = (df_clean['price_clean'] < 0) | (df_clean['price_clean'] > 500)
print(f"   Invalid prices: {invalid_price.sum()}")

# Rating should be 0-5
invalid_rating = (df_numerical['rating'] < 0) | (df_numerical['rating'] > 5)
print(f"   Invalid ratings: {invalid_rating.sum()}")
if invalid_rating.sum() > 0:
    print("   Invalid rating values:")
    print(df_numerical[invalid_rating][['product_id', 'rating']])

# Discount should be 0-100%
invalid_discount = (df_clean['discount_numeric'] < 0) | (df_clean['discount_numeric'] > 100)
print(f"   Invalid discounts: {invalid_discount.sum()}")
if invalid_discount.sum() > 0:
    print("   Invalid discount values:")
    print(df_clean[invalid_discount][['product_id', 'discount_pct', 'discount_numeric']])

# 5. Handle negative values
print("\n5. Handling Negative Values:")

# Quantity should be non-negative
negative_qty = df_clean['quantity'] < 0
print(f"   Negative quantities: {negative_qty.sum()}")

# Method 1: Replace with 0
df_clean['quantity_clean'] = df_clean['quantity'].clip(lower=0)

# Method 2: Take absolute value (if negative means return)
df_clean['quantity_abs'] = df_clean['quantity'].abs()

print("   Quantity handling:")
print(df_clean[['quantity', 'quantity_clean', 'quantity_abs']].head(10))

# 6. Standardize units (weight to kg)
print("\n6. Standardizing Units (Weight to kg):")

def convert_weight_to_kg(weight):
    """Convert various weight formats to kg"""
    weight_str = str(weight).lower().strip()
    
    # Extract number
    numbers = re.findall(r'\d+\.?\d*', weight_str)
    if not numbers:
        return np.nan
    
    value = float(numbers[0])
    
    # Convert based on unit
    if 'g' in weight_str and 'kg' not in weight_str:
        value = value / 1000  # grams to kg
    elif 'lb' in weight_str or 'pound' in weight_str:
        value = value * 0.453592  # pounds to kg
    elif 'oz' in weight_str:
        value = value * 0.0283495  # ounces to kg
    
    return round(value, 3)

df_clean['weight_kg'] = df_clean['weight'].apply(convert_weight_to_kg)

print("   Weight standardization:")
for orig, converted in zip(df_numerical['weight'], df_clean['weight_kg']):
    print(f"   {orig:10} → {converted} kg")

# 7. Round and control precision
print("\n7. Controlling Precision:")

# Round price to 2 decimals
df_clean['price_rounded'] = df_clean['price_clean'].round(2)

# Round rating to 1 decimal
df_clean['rating_clean'] = pd.to_numeric(df_numerical['rating'], errors='coerce')
df_clean['rating_clean'] = df_clean['rating_clean'].clip(0, 5).round(1)

print("   Precision control:")
print(df_clean[['price_clean', 'price_rounded', 'rating_clean']].head(10))

# 8. Bin continuous variables
print("\n8. Binning Continuous Variables:")

# Bin prices into categories
price_bins = [0, 30, 60, 90, 150]
price_labels = ['Budget', 'Mid-Range', 'Premium', 'Luxury']
df_clean['price_category'] = pd.cut(df_clean['price_rounded'], bins=price_bins, labels=price_labels)

print("   Price categories:")
print(df_clean[['price_rounded', 'price_category']].head(10))
print("\n   Price category distribution:")
print(df_clean['price_category'].value_counts().sort_index())

# Bin ratings
rating_bins = [0, 2, 3, 4, 5]
rating_labels = ['Poor', 'Fair', 'Good', 'Excellent']
df_clean['rating_category'] = pd.cut(df_clean['rating_clean'], bins=rating_bins, labels=rating_labels)

print("\n   Rating categories:")
print(df_clean['rating_category'].value_counts().sort_index())

# 9. Handle extreme outliers
print("\n9. Handling Extreme Outliers:")

# Identify extreme quantities
q1 = df_clean['quantity_clean'].quantile(0.25)
q3 = df_clean['quantity_clean'].quantile(0.75)
iqr = q3 - q1
lower_bound = q1 - 3 * iqr  # 3 IQR for extreme outliers
upper_bound = q3 + 3 * iqr

extreme_outliers = (df_clean['quantity_clean'] < lower_bound) | (df_clean['quantity_clean'] > upper_bound)
print(f"   Extreme quantity outliers: {extreme_outliers.sum()}")

if extreme_outliers.sum() > 0:
    print("   Extreme values:")
    print(df_clean[extreme_outliers][['product_id', 'quantity_clean']])

# Cap extreme values
df_clean['quantity_capped'] = df_clean['quantity_clean'].clip(lower=lower_bound, upper=upper_bound)

print(f"\n   Capped at: {lower_bound:.0f} - {upper_bound:.0f}")

# 10. Handle zeros
print("\n10. Handling Zero Values:")

zero_price = (df_clean['price_rounded'] == 0).sum()
zero_qty = (df_clean['quantity_clean'] == 0).sum()
zero_rating = (df_clean['rating_clean'] == 0).sum()

print(f"   Zero prices: {zero_price}")
print(f"   Zero quantities: {zero_qty}")
print(f"   Zero ratings: {zero_rating}")

# Replace zero prices with minimum non-zero price
min_nonzero_price = df_clean[df_clean['price_rounded'] > 0]['price_rounded'].min()
df_clean['price_final'] = df_clean['price_rounded'].replace(0, min_nonzero_price)

print(f"   Replaced zero prices with minimum: ${min_nonzero_price:.2f}")

# 11. Create derived numeric features
print("\n11. Creating Derived Numeric Features:")

# Calculate discounted price
df_clean['discount_decimal'] = df_clean['discount_numeric'] / 100
df_clean['discounted_price'] = df_clean['price_final'] * (1 - df_clean['discount_decimal'])
df_clean['savings'] = df_clean['price_final'] - df_clean['discounted_price']

print("   Price calculations:")
print(df_clean[['price_final', 'discount_numeric', 'discounted_price', 'savings']].head(8))

# Calculate total value
df_clean['total_value'] = df_clean['discounted_price'] * df_clean['quantity_clean']

print("\n   Total value per product:")
print(df_clean[['product_id', 'quantity_clean', 'discounted_price', 'total_value']].head(8))

# 12. Normalize and compare
print("\n12. Normalization Examples:")

# Min-max normalization (0-1)
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
df_clean['price_normalized'] = scaler.fit_transform(df_clean[['price_final']])

# Z-score normalization
df_clean['price_zscore'] = (df_clean['price_final'] - df_clean['price_final'].mean()) / df_clean['price_final'].std()

print("   Price normalization:")
print(df_clean[['price_final', 'price_normalized', 'price_zscore']].head(8))

# 13. Summary statistics
print("\n13. Summary Statistics:")

numeric_cols = ['price_final', 'quantity_clean', 'weight_kg', 'rating_clean', 'discount_numeric']
summary_stats = df_clean[numeric_cols].describe()

print(summary_stats)

# 14. Final validation
print("\n14. Final Data Validation:")

validation_results = {
    'Column': [],
    'Min': [],
    'Max': [],
    'Mean': [],
    'Missing': [],
    'Invalid': []
}

for col in numeric_cols:
    validation_results['Column'].append(col)
    validation_results['Min'].append(f"{df_clean[col].min():.2f}")
    validation_results['Max'].append(f"{df_clean[col].max():.2f}")
    validation_results['Mean'].append(f"{df_clean[col].mean():.2f}")
    validation_results['Missing'].append(df_clean[col].isna().sum())
    
    # Count invalid based on expected ranges
    if col == 'rating_clean':
        invalid = ((df_clean[col] < 0) | (df_clean[col] > 5)).sum()
    elif col == 'discount_numeric':
        invalid = ((df_clean[col] < 0) | (df_clean[col] > 100)).sum()
    else:
        invalid = (df_clean[col] < 0).sum()
    
    validation_results['Invalid'].append(invalid)

validation_df = pd.DataFrame(validation_results)
print(validation_df.to_string(index=False))

print("\n   ✓ Numerical data cleaning complete!")

## 12. Text Data Cleaning

**What is it?**
Text data cleaning involves preprocessing and standardizing unstructured text for analysis, NLP, or machine learning.

**Why use it?**
- Improves text analysis accuracy
- Reduces noise in text data
- Standardizes text format
- Prepares data for NLP models
- Removes irrelevant information
- Handles encoding issues

**When to use it?**
- Working with user-generated content
- Natural Language Processing (NLP)
- Text mining and sentiment analysis
- Search and information retrieval
- Text classification tasks
- Topic modeling

**Common text cleaning operations:**
1. **Lowercasing**: Standardize case
2. **Removing punctuation**: Clean special characters
3. **Removing stopwords**: Filter common words
4. **Tokenization**: Split into words/sentences
5. **Stemming/Lemmatization**: Reduce words to root form
6. **Removing HTML/URLs**: Clean web content
7. **Handling emojis/special characters**
8. **Spell checking and correction**

In [ ]:
# ============================================
# 12. TEXT DATA CLEANING
# ============================================

import string

# Create sample text data with various issues
text_data = {
    'review_id': range(1, 11),
    'review_text': [
        'GREAT product!!! Really LOVE it!!! 😍😍😍',
        '   Bad quality...    disappointing ☹️  ',
        'Visit our website: https://example.com for more info!!!',
        'Email us at support@example.com for help',
        '<p>HTML tags should be <b>removed</b></p>',
        'Too many    spaces   and\n\nnewlines\n',
        'Misspelled: recieve, occured, definately',
        'Abbreviations: u r so gr8! lol omg btw',
        '$$$SPECIAL OFFER$$$ Buy NOW!!! 50% OFF!!!',
        'Mixed languages: This is English. C\'est français. これは日本語です。'
    ],
    'comment': [
        'Good product',
        'Not recommended...',
        'Check website',
        'Contact support',
        'HTML content',
        'Whitespace issues',
        'Spelling errors',
        'SMS language',
        'Spam content',
        'Multilingual'
    ]
}

df_text = pd.DataFrame(text_data)

print("=" * 80)
print("TEXT DATA CLEANING")
print("=" * 80)
print("\n1. Original Text Data:")
for idx, row in df_text.iterrows():
    print(f"\n{idx + 1}. {row['comment']}:")
    print(f"   '{row['review_text']}'")

# 2. Basic text cleaning
print("\n" + "=" * 80)
print("2. Basic Text Cleaning:")

df_clean = df_text.copy()

# Lowercase
df_clean['text_lower'] = df_clean['review_text'].str.lower()

# Strip whitespace
df_clean['text_stripped'] = df_clean['text_lower'].str.strip()

print("   After lowercase and strip:")
print(df_clean[['comment', 'text_stripped']].head(5))

# 3. Remove URLs
print("\n3. Removing URLs:")

def remove_urls(text):
    """Remove URLs from text"""
    # Pattern for http/https URLs
    url_pattern = r'https?://\S+|www\.\S+'
    return re.sub(url_pattern, '', text)

df_clean['text_no_urls'] = df_clean['text_stripped'].apply(remove_urls)

print("   Before → After:")
for orig, cleaned in zip(df_clean['text_stripped'].head(5), df_clean['text_no_urls'].head(5)):
    if orig != cleaned:
        print(f"   '{orig}'")
        print(f"   → '{cleaned}'")

# 4. Remove emails
print("\n4. Removing Email Addresses:")

def remove_emails(text):
    """Remove email addresses from text"""
    email_pattern = r'\S+@\S+'
    return re.sub(email_pattern, '', text)

df_clean['text_no_emails'] = df_clean['text_no_urls'].apply(remove_emails)

print("   Before → After:")
for orig, cleaned in zip(df_clean['text_no_urls'][3:5], df_clean['text_no_emails'][3:5]):
    if orig != cleaned:
        print(f"   '{orig}'")
        print(f"   → '{cleaned}'")

# 5. Remove HTML tags
print("\n5. Removing HTML Tags:")

def remove_html(text):
    """Remove HTML tags from text"""
    html_pattern = r'<[^>]+>'
    return re.sub(html_pattern, '', text)

df_clean['text_no_html'] = df_clean['text_no_emails'].apply(remove_html)

print("   Before → After:")
for orig, cleaned in zip(df_clean['text_no_emails'][4:6], df_clean['text_no_html'][4:6]):
    if orig != cleaned:
        print(f"   '{orig}'")
        print(f"   → '{cleaned}'")

# 6. Remove extra whitespace
print("\n6. Removing Extra Whitespace:")

def remove_extra_whitespace(text):
    """Remove extra spaces, tabs, newlines"""
    # Replace multiple spaces with single space
    text = re.sub(r'\s+', ' ', text)
    # Remove leading/trailing whitespace
    return text.strip()

df_clean['text_clean_ws'] = df_clean['text_no_html'].apply(remove_extra_whitespace)

print("   Before → After:")
for orig, cleaned in zip(df_clean['text_no_html'][5:7], df_clean['text_clean_ws'][5:7]):
    if orig != cleaned:
        print(f"   '{orig}'")
        print(f"   → '{cleaned}'")

# 7. Remove punctuation
print("\n7. Removing Punctuation:")

def remove_punctuation(text):
    """Remove punctuation from text"""
    return text.translate(str.maketrans('', '', string.punctuation))

df_clean['text_no_punct'] = df_clean['text_clean_ws'].apply(remove_punctuation)

print("   Sample with punctuation removed:")
print(df_clean[['comment', 'text_no_punct']].head(5))

# 8. Remove numbers
print("\n8. Removing Numbers:")

def remove_numbers(text):
    """Remove numbers from text"""
    return re.sub(r'\d+', '', text)

df_clean['text_no_numbers'] = df_clean['text_no_punct'].apply(remove_numbers)

print("   After removing numbers:")
print(df_clean[['comment', 'text_no_numbers']].head(5))

# 9. Remove emojis
print("\n9. Removing Emojis and Special Characters:")

def remove_emojis(text):
    """Remove emojis and special characters"""
    # Pattern for emojis
    emoji_pattern = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags
        u"\U00002702-\U000027B0"
        u"\U000024C2-\U0001F251"
        "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', text)

df_clean['text_no_emojis'] = df_clean['text_clean_ws'].apply(remove_emojis)

print("   Before → After:")
for idx in [0, 1]:
    print(f"   '{df_clean['text_clean_ws'].iloc[idx]}'")
    print(f"   → '{df_clean['text_no_emojis'].iloc[idx]}'")

# 10. Expand abbreviations
print("\n10. Expanding Common Abbreviations:")

abbreviations = {
    ' u ': ' you ',
    ' r ': ' are ',
    ' gr8 ': ' great ',
    ' lol ': ' laughing out loud ',
    ' omg ': ' oh my god ',
    ' btw ': ' by the way ',
    ' asap ': ' as soon as possible ',
    ' fyi ': ' for your information ',
}

def expand_abbreviations(text, abbrev_dict):
    """Expand common abbreviations"""
    text = ' ' + text + ' '  # Add spaces for boundary matching
    for abbrev, full in abbrev_dict.items():
        text = text.replace(abbrev, full)
    return text.strip()

df_clean['text_expanded'] = df_clean['text_no_punct'].apply(
    lambda x: expand_abbreviations(x, abbreviations)
)

print("   Before → After:")
print(f"   '{df_clean['text_no_punct'].iloc[7]}'")
print(f"   → '{df_clean['text_expanded'].iloc[7]}'")

# 11. Remove stopwords
print("\n11. Removing Stopwords:")

# Common English stopwords
stopwords = {
    'a', 'an', 'and', 'are', 'as', 'at', 'be', 'by', 'for', 'from',
    'has', 'he', 'in', 'is', 'it', 'its', 'of', 'on', 'that', 'the',
    'to', 'was', 'will', 'with', 'this', 'but', 'they', 'have', 'had',
    'what', 'when', 'where', 'who', 'which', 'why', 'how'
}

def remove_stopwords(text, stopwords_set):
    """Remove stopwords from text"""
    words = text.split()
    filtered_words = [word for word in words if word not in stopwords_set]
    return ' '.join(filtered_words)

df_clean['text_no_stopwords'] = df_clean['text_no_numbers'].apply(
    lambda x: remove_stopwords(x, stopwords)
)

print("   Before → After:")
for idx in [0, 1]:
    print(f"   '{df_clean['text_no_numbers'].iloc[idx]}'")
    print(f"   → '{df_clean['text_no_stopwords'].iloc[idx]}'")

# 12. Tokenization
print("\n12. Tokenization:")

def tokenize_text(text):
    """Split text into tokens (words)"""
    return text.split()

df_clean['tokens'] = df_clean['text_no_stopwords'].apply(tokenize_text)

print("   Tokenized text (first 3):")
for idx in range(3):
    print(f"   {idx + 1}. {df_clean['tokens'].iloc[idx]}")

# 13. Count features
print("\n13. Text Feature Extraction:")

# Character count
df_clean['char_count'] = df_clean['review_text'].str.len()

# Word count
df_clean['word_count'] = df_clean['review_text'].str.split().str.len()

# Unique word count
df_clean['unique_word_count'] = df_clean['tokens'].apply(lambda x: len(set(x)))

# Average word length
df_clean['avg_word_length'] = df_clean['review_text'].apply(
    lambda x: np.mean([len(word) for word in x.split()] if x.split() else [0])
)

print("   Text statistics:")
print(df_clean[['comment', 'char_count', 'word_count', 'unique_word_count', 'avg_word_length']].head(8))

# 14. Detect language/encoding issues
print("\n14. Handling Encoding Issues:")

def keep_ascii_only(text):
    """Keep only ASCII characters"""
    return ''.join(char for char in text if ord(char) < 128)

df_clean['text_ascii'] = df_clean['text_clean_ws'].apply(keep_ascii_only)

print("   Before → After (non-ASCII removal):")
print(f"   '{df_clean['text_clean_ws'].iloc[9]}'")
print(f"   → '{df_clean['text_ascii'].iloc[9]}'")

# 15. Create comprehensive text cleaning pipeline
print("\n15. Comprehensive Text Cleaning Pipeline:")

def clean_text_pipeline(text, 
                        lowercase=True,
                        remove_url=True,
                        remove_email=True,
                        remove_html_tags=True,
                        remove_punct=True,
                        remove_num=True,
                        remove_emoji=True,
                        remove_extra_ws=True,
                        remove_stops=False):
    """
    Comprehensive text cleaning pipeline
    
    Parameters:
    - text: Input text string
    - lowercase: Convert to lowercase
    - remove_url: Remove URLs
    - remove_email: Remove email addresses
    - remove_html_tags: Remove HTML tags
    - remove_punct: Remove punctuation
    - remove_num: Remove numbers
    - remove_emoji: Remove emojis
    - remove_extra_ws: Remove extra whitespace
    - remove_stops: Remove stopwords
    """
    if pd.isna(text):
        return ''
    
    # Lowercase
    if lowercase:
        text = text.lower()
    
    # Remove URLs
    if remove_url:
        text = remove_urls(text)
    
    # Remove emails
    if remove_email:
        text = remove_emails(text)
    
    # Remove HTML
    if remove_html_tags:
        text = remove_html(text)
    
    # Remove emojis
    if remove_emoji:
        text = remove_emojis(text)
    
    # Remove punctuation
    if remove_punct:
        text = remove_punctuation(text)
    
    # Remove numbers
    if remove_num:
        text = remove_numbers(text)
    
    # Remove stopwords
    if remove_stops:
        text = remove_stopwords(text, stopwords)
    
    # Remove extra whitespace
    if remove_extra_ws:
        text = remove_extra_whitespace(text)
    
    return text

# Apply pipeline
df_clean['text_final'] = df_clean['review_text'].apply(clean_text_pipeline)

print("\n   Original → Final Cleaned Text:")
for idx in range(len(df_clean)):
    print(f"\n   {idx + 1}. {df_clean['comment'].iloc[idx]}:")
    print(f"   Original: '{df_clean['review_text'].iloc[idx]}'")
    print(f"   Cleaned:  '{df_clean['text_final'].iloc[idx]}'")

# 16. Summary
print("\n" + "=" * 80)
print("16. Text Cleaning Summary:")

summary = pd.DataFrame({
    'Operation': [
        'Lowercase',
        'Remove URLs',
        'Remove Emails',
        'Remove HTML',
        'Remove Emojis',
        'Remove Punctuation',
        'Remove Numbers',
        'Remove Extra Whitespace',
        'Remove Stopwords',
        'Tokenization'
    ],
    'Purpose': [
        'Standardize case',
        'Remove web links',
        'Remove email addresses',
        'Clean HTML content',
        'Remove emoji characters',
        'Remove special characters',
        'Remove numeric values',
        'Clean spacing',
        'Remove common words',
        'Split into words'
    ],
    'Recommended_For': [
        'All text analysis',
        'User content, reviews',
        'User content',
        'Web-scraped data',
        'Social media data',
        'NLP, topic modeling',
        'Text classification',
        'All text analysis',
        'NLP, keyword extraction',
        'NLP, word analysis'
    ]
})

print(summary.to_string(index=False))

print("\n   ✓ Text data cleaning complete!")

## 13. Standardization & Normalization

**What is it?**
Standardization and normalization are techniques to scale numeric data to a common range or distribution.

**Why use it?**
- Makes features comparable
- Improves ML model performance
- Required for distance-based algorithms
- Speeds up gradient descent
- Prevents features from dominating
- Handles different units/scales

**When to use it?**
- Before machine learning (especially SVM, KNN, neural networks)
- Features have different scales
- Distance-based algorithms
- Gradient descent optimization
- Comparing variables with different units
- PCA and clustering

**Common techniques:**
1. **Min-Max Scaling**: Scale to [0, 1] or custom range
2. **Z-Score (Standardization)**: Mean=0, Std=1
3. **Robust Scaling**: Uses median and IQR (outlier-resistant)
4. **Log Transformation**: Handle skewed data
5. **Box-Cox**: Power transformation
6. **Unit Vector**: Scale to unit norm

In [ ]:
# ============================================
# 13. STANDARDIZATION & NORMALIZATION
# ============================================

from sklearn.preprocessing import MinMaxScaler, StandardScaler, RobustScaler, Normalizer
from scipy import stats

# Create sample data with different scales
np.random.seed(42)
scaling_data = {
    'age': np.random.randint(18, 80, 20),
    'salary': np.random.randint(30000, 150000, 20),
    'years_experience': np.random.randint(0, 30, 20),
    'rating': np.random.uniform(1, 5, 20),
    'transactions': np.random.randint(1, 1000, 20)
}

df_scaling = pd.DataFrame(scaling_data)

print("=" * 80)
print("STANDARDIZATION & NORMALIZATION")
print("=" * 80)
print("\n1. Original Data (Different Scales):")
print(df_scaling.head(10))

print("\n   Descriptive Statistics:")
print(df_scaling.describe())

# 2. Min-Max Scaling (Normalization)
print("\n2. Min-Max Scaling (0 to 1):")

scaler_minmax = MinMaxScaler()
df_minmax = pd.DataFrame(
    scaler_minmax.fit_transform(df_scaling),
    columns=[f'{col}_minmax' for col in df_scaling.columns]
)

print("   Formula: X_scaled = (X - X_min) / (X_max - X_min)")
print("\n   Min-Max scaled data:")
print(df_minmax.head(10))

print("\n   Min-Max statistics:")
print(df_minmax.describe())

# Visualize before and after
fig, axes = plt.subplots(2, 3, figsize=(15, 8))

# Original distributions
for idx, col in enumerate(['age', 'salary', 'years_experience']):
    df_scaling[col].hist(bins=15, ax=axes[0, idx], edgecolor='black')
    axes[0, idx].set_title(f'Original: {col}')
    axes[0, idx].set_xlabel(col)
    axes[0, idx].set_ylabel('Frequency')

# Min-Max scaled distributions
for idx, col in enumerate(['age', 'salary', 'years_experience']):
    df_minmax[f'{col}_minmax'].hist(bins=15, ax=axes[1, idx], edgecolor='black', color='green')
    axes[1, idx].set_title(f'Min-Max Scaled: {col}')
    axes[1, idx].set_xlabel(f'{col} (scaled)')
    axes[1, idx].set_ylabel('Frequency')

plt.tight_layout()
plt.show()

# 3. Custom range Min-Max Scaling
print("\n3. Min-Max Scaling (Custom Range: -1 to 1):")

scaler_custom = MinMaxScaler(feature_range=(-1, 1))
df_custom = pd.DataFrame(
    scaler_custom.fit_transform(df_scaling),
    columns=[f'{col}_custom' for col in df_scaling.columns]
)

print("   Scaled to [-1, 1] range:")
print(df_custom.head(10))
print(f"\n   Min: {df_custom.min().min():.2f}, Max: {df_custom.max().max():.2f}")

# 4. Z-Score Standardization
print("\n4. Z-Score Standardization (StandardScaler):")

scaler_standard = StandardScaler()
df_standard = pd.DataFrame(
    scaler_standard.fit_transform(df_scaling),
    columns=[f'{col}_std' for col in df_scaling.columns]
)

print("   Formula: X_scaled = (X - mean) / std_dev")
print("\n   Standardized data:")
print(df_standard.head(10))

print("\n   Standardized statistics (mean ≈ 0, std ≈ 1):")
print(df_standard.describe())

# Visualize Z-score transformation
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Salary before
df_scaling['salary'].hist(bins=15, ax=axes[0], edgecolor='black')
axes[0].set_title('Original Salary Distribution')
axes[0].set_xlabel('Salary')
axes[0].axvline(df_scaling['salary'].mean(), color='red', linestyle='--', label='Mean')
axes[0].legend()

# Salary after
df_standard['salary_std'].hist(bins=15, ax=axes[1], edgecolor='black', color='green')
axes[1].set_title('Standardized Salary (Z-Score)')
axes[1].set_xlabel('Salary (z-score)')
axes[1].axvline(0, color='red', linestyle='--', label='Mean = 0')
axes[1].legend()

plt.tight_layout()
plt.show()

# 5. Robust Scaling
print("\n5. Robust Scaling (Outlier-Resistant):")

# Add some outliers
df_with_outliers = df_scaling.copy()
df_with_outliers.loc[0, 'salary'] = 500000  # Outlier

scaler_robust = RobustScaler()
df_robust = pd.DataFrame(
    scaler_robust.fit_transform(df_with_outliers),
    columns=[f'{col}_robust' for col in df_scaling.columns]
)

print("   Formula: X_scaled = (X - median) / IQR")
print("   Uses median and IQR instead of mean and std")
print("\n   Robust scaled data (with outlier):")
print(df_robust.head(10))

# Compare StandardScaler vs RobustScaler with outliers
df_standard_outlier = pd.DataFrame(
    scaler_standard.fit_transform(df_with_outliers),
    columns=[f'{col}_std' for col in df_scaling.columns]
)

print("\n   Comparison with outlier:")
print("   StandardScaler:")
print(f"   Salary mean: {df_standard_outlier['salary_std'].mean():.2f}")
print(f"   Salary std: {df_standard_outlier['salary_std'].std():.2f}")
print("\n   RobustScaler:")
print(f"   Salary median: {df_robust['salary_robust'].median():.2f}")
print(f"   Salary IQR: {df_robust['salary_robust'].quantile(0.75) - df_robust['salary_robust'].quantile(0.25):.2f}")

# 6. Log Transformation
print("\n6. Log Transformation (for Skewed Data):")

# Create skewed data
skewed_data = {
    'income': [20000, 25000, 30000, 35000, 40000, 50000, 60000, 80000, 120000, 500000]
}
df_skewed = pd.DataFrame(skewed_data)

# Apply log transformation
df_skewed['income_log'] = np.log1p(df_skewed['income'])  # log1p = log(1 + x)

# Calculate skewness
skew_before = stats.skew(df_skewed['income'])
skew_after = stats.skew(df_skewed['income_log'])

print(f"   Skewness before log: {skew_before:.2f}")
print(f"   Skewness after log: {skew_after:.2f}")

# Visualize
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

df_skewed['income'].hist(bins=10, ax=axes[0], edgecolor='black')
axes[0].set_title(f'Original Income (Skew: {skew_before:.2f})')
axes[0].set_xlabel('Income')

df_skewed['income_log'].hist(bins=10, ax=axes[1], edgecolor='black', color='green')
axes[1].set_title(f'Log-Transformed Income (Skew: {skew_after:.2f})')
axes[1].set_xlabel('Log(Income)')

plt.tight_layout()
plt.show()

# 7. Box-Cox Transformation
print("\n7. Box-Cox Transformation:")

# Box-Cox requires positive values
positive_data = df_scaling['age'].copy()

# Apply Box-Cox
transformed_data, lambda_param = stats.boxcox(positive_data)

print(f"   Optimal lambda: {lambda_param:.4f}")
print(f"   Skewness before: {stats.skew(positive_data):.4f}")
print(f"   Skewness after: {stats.skew(transformed_data):.4f}")

# 8. Unit Vector Normalization (L2 Norm)
print("\n8. Unit Vector Normalization (L2 Norm):")

normalizer = Normalizer(norm='l2')
df_normalized = pd.DataFrame(
    normalizer.fit_transform(df_scaling),
    columns=[f'{col}_l2' for col in df_scaling.columns]
)

print("   Formula: X_scaled = X / ||X||_2")
print("   Each row scaled to unit length")
print("\n   L2 normalized data:")
print(df_normalized.head(10))

# Check row norms (should be 1)
row_norms = np.linalg.norm(df_normalized.values, axis=1)
print(f"\n   Row norms (should be 1.0): {row_norms[:5]}")

# 9. Compare all scaling methods
print("\n9. Comparing All Scaling Methods:")

sample_idx = 0
original_values = df_scaling.iloc[sample_idx]

comparison = pd.DataFrame({
    'Feature': df_scaling.columns,
    'Original': original_values.values,
    'Min-Max [0,1]': df_minmax.iloc[sample_idx].values,
    'Min-Max [-1,1]': df_custom.iloc[sample_idx].values,
    'Z-Score': df_standard.iloc[sample_idx].values,
    'Robust': df_robust.iloc[sample_idx].values,
    'L2 Norm': df_normalized.iloc[sample_idx].values
})

print(f"\n   Sample row {sample_idx} across all scaling methods:")
print(comparison.to_string(index=False))

# 10. When to use each method
print("\n10. Scaling Method Selection Guide:")

guide = pd.DataFrame({
    'Method': [
        'Min-Max [0,1]',
        'Min-Max [a,b]',
        'Z-Score (Standard)',
        'Robust Scaler',
        'Log Transform',
        'Box-Cox',
        'L2 Normalization'
    ],
    'Best_For': [
        'Neural networks, image data',
        'Bounded range requirements',
        'Normally distributed data, linear models',
        'Data with outliers',
        'Right-skewed data (income, sales)',
        'Severely skewed data',
        'Text vectorization, cosine similarity'
    ],
    'Preserves': [
        'Relationships',
        'Relationships',
        'Relationships (z-scores)',
        'Relationships (outlier-resistant)',
        'Order (reduces skew)',
        'Order (optimizes normality)',
        'Direction (unit vector)'
    ],
    'Sensitive_to_Outliers': [
        'Yes',
        'Yes',
        'Yes',
        'No',
        'Partially',
        'Partially',
        'Yes'
    ]
})

print(guide.to_string(index=False))

# 11. Inverse transformation
print("\n11. Inverse Transformation (Reversing Scaling):")

# Scale data
scaled_salary = scaler_minmax.fit_transform(df_scaling[['salary']])

# Inverse transform (get back original values)
original_salary = scaler_minmax.inverse_transform(scaled_salary)

print("   Original → Scaled → Inverse:")
for i in range(5):
    print(f"   ${df_scaling['salary'].iloc[i]:,} → {scaled_salary[i][0]:.4f} → ${original_salary[i][0]:,.0f}")

# 12. Handling new data (transform only, no fit)
print("\n12. Scaling New Data (Transform Only):")

# New unseen data
new_data = pd.DataFrame({
    'age': [25, 45],
    'salary': [50000, 100000],
    'years_experience': [2, 15],
    'rating': [3.5, 4.8],
    'transactions': [100, 500]
})

print("   New data:")
print(new_data)

# Transform using already fitted scaler (don't fit again!)
new_scaled = scaler_minmax.transform(new_data)
new_scaled_df = pd.DataFrame(
    new_scaled,
    columns=[f'{col}_scaled' for col in new_data.columns]
)

print("\n   New data scaled with existing scaler:")
print(new_scaled_df)

# 13. Summary
print("\n13. Standardization & Normalization Summary:")

print("\n   Key Differences:")
print("   • Normalization (Min-Max): Scales to fixed range [0,1] or [a,b]")
print("   • Standardization (Z-Score): Centers around 0 with unit variance")
print("   • Robust Scaling: Like standardization but uses median/IQR")
print("   • Log/Box-Cox: Transforms distribution shape (handles skewness)")
print("   • L2 Normalization: Scales each sample to unit norm")

print("\n   Best Practices:")
print("   • Always fit on training data only, then transform test data")
print("   • Save scalers for production use (inverse_transform, new data)")
print("   • Choose based on data distribution and algorithm requirements")
print("   • Use Robust Scaler when outliers are present")
print("   • Use log/Box-Cox for skewed data before other scaling")

print("\n   ✓ Standardization & normalization complete!")

## 14. Advanced Data Cleaning Techniques

**What is it?**
Advanced techniques combine multiple cleaning methods and use sophisticated approaches for complex data quality issues.

**Why use it?**
- Handles complex data quality problems
- Automates cleaning workflows
- Improves accuracy beyond basic methods
- Scales to large datasets
- Detects subtle patterns and anomalies
- Maintains data relationships

**When to use it?**
- Basic cleaning insufficient
- Complex data quality issues
- Large-scale data processing
- Need automated solutions
- Multiple related cleaning tasks
- Production pipelines

**Advanced techniques:**
1. **Fuzzy matching**: Match similar but not identical strings
2. **Automated anomaly detection**: ML-based outlier detection
3. **Advanced imputation**: KNN, MICE, regression-based
4. **Deduplication**: Record linkage, entity resolution
5. **Data profiling**: Automated quality assessment
6. **Constraint-based validation**: Complex business rules

In [ ]:
# ============================================
# 14. ADVANCED DATA CLEANING TECHNIQUES
# ============================================

from sklearn.impute import KNNImputer
from sklearn.ensemble import IsolationForest
from difflib import SequenceMatcher

print("=" * 80)
print("ADVANCED DATA CLEANING TECHNIQUES")
print("=" * 80)

# 1. Fuzzy String Matching
print("\n1. Fuzzy String Matching:")

# Sample data with similar but not identical strings
companies = [
    'Microsoft Corporation',
    'Microsoft Corp',
    'Microsft Corporation',  # Typo
    'Google LLC',
    'Google Inc',
    'Gogle LLC',  # Typo
    'Amazon.com Inc',
    'Amazon Inc',
    'Apple Inc.',
    'Apple Incorporated'
]

def fuzzy_similarity(str1, str2):
    """Calculate similarity ratio between two strings"""
    return SequenceMatcher(None, str1.lower(), str2.lower()).ratio()

def find_fuzzy_matches(string, string_list, threshold=0.8):
    """Find similar strings above similarity threshold"""
    matches = []
    for s in string_list:
        similarity = fuzzy_similarity(string, s)
        if similarity >= threshold and s != string:
            matches.append((s, similarity))
    return sorted(matches, key=lambda x: x[1], reverse=True)

print("   Fuzzy matching examples (threshold=0.8):")
test_strings = ['Microsoft Corporation', 'Google LLC', 'Amazon Inc']
for test in test_strings:
    matches = find_fuzzy_matches(test, companies, threshold=0.8)
    if matches:
        print(f"\n   '{test}' matches:")
        for match, score in matches:
            print(f"     - '{match}' (similarity: {score:.2f})")

# Standardize using fuzzy matching
def standardize_with_fuzzy(name, canonical_names, threshold=0.85):
    """Standardize name using fuzzy matching to canonical list"""
    best_match = None
    best_score = 0
    
    for canonical in canonical_names:
        score = fuzzy_similarity(name, canonical)
        if score > best_score and score >= threshold:
            best_score = score
            best_match = canonical
    
    return best_match if best_match else name

canonical_companies = ['Microsoft Corporation', 'Google LLC', 'Amazon.com Inc', 'Apple Inc.']

print("\n   Standardization using fuzzy matching:")
for company in companies:
    standardized = standardize_with_fuzzy(company, canonical_companies, threshold=0.75)
    if company != standardized:
        print(f"   '{company}' → '{standardized}'")

# 2. Advanced Imputation (KNN)
print("\n2. Advanced Imputation (K-Nearest Neighbors):")

# Create data with missing values
impute_data = {
    'age': [25, 30, np.nan, 40, 45, np.nan, 35, 50, 28, np.nan],
    'salary': [50000, 60000, 55000, np.nan, 80000, 70000, np.nan, 90000, 52000, 58000],
    'experience': [2, 5, 3, np.nan, 12, 8, 6, np.nan, 3, 4]
}

df_impute = pd.DataFrame(impute_data)

print("   Original data with missing values:")
print(df_impute)

# KNN Imputation (fills based on k nearest neighbors)
knn_imputer = KNNImputer(n_neighbors=3)
df_knn_imputed = pd.DataFrame(
    knn_imputer.fit_transform(df_impute),
    columns=df_impute.columns
)

print("\n   After KNN imputation (k=3):")
print(df_knn_imputed)

# Compare with simple mean imputation
df_mean_imputed = df_impute.fillna(df_impute.mean())

print("\n   Comparison (KNN vs Mean imputation):")
for idx in df_impute[df_impute.isna().any(axis=1)].index:
    print(f"\n   Row {idx}:")
    print(f"   Original: {df_impute.iloc[idx].to_dict()}")
    print(f"   KNN:      {df_knn_imputed.iloc[idx].round(0).to_dict()}")
    print(f"   Mean:     {df_mean_imputed.iloc[idx].round(0).to_dict()}")

# 3. Automated Anomaly Detection (Isolation Forest)
print("\n3. Automated Anomaly Detection (Isolation Forest):")

# Create data with anomalies
np.random.seed(42)
normal_data = np.random.normal(50, 10, 95)
anomalies = np.array([5, 10, 90, 95, 100])  # Clear outliers
all_data = np.concatenate([normal_data, anomalies])

df_anomaly = pd.DataFrame({
    'value': all_data,
    'feature2': np.random.normal(100, 20, 100)
})

# Train Isolation Forest
iso_forest = IsolationForest(contamination=0.05, random_state=42)
predictions = iso_forest.fit_predict(df_anomaly)

df_anomaly['is_anomaly'] = predictions == -1

print(f"   Total records: {len(df_anomaly)}")
print(f"   Detected anomalies: {df_anomaly['is_anomaly'].sum()}")

print("\n   Detected anomalies:")
anomalies_df = df_anomaly[df_anomaly['is_anomaly']].sort_values('value')
print(anomalies_df)

# Visualize
plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
plt.scatter(df_anomaly[~df_anomaly['is_anomaly']]['value'],
           df_anomaly[~df_anomaly['is_anomaly']]['feature2'],
           c='blue', label='Normal', alpha=0.6)
plt.scatter(df_anomaly[df_anomaly['is_anomaly']]['value'],
           df_anomaly[df_anomaly['is_anomaly']]['feature2'],
           c='red', label='Anomaly', s=100, marker='x')
plt.xlabel('Value')
plt.ylabel('Feature 2')
plt.title('Isolation Forest Anomaly Detection')
plt.legend()

plt.subplot(1, 2, 2)
df_anomaly['value'].hist(bins=30, edgecolor='black', alpha=0.7)
for val in df_anomaly[df_anomaly['is_anomaly']]['value']:
    plt.axvline(val, color='red', linestyle='--', alpha=0.7)
plt.xlabel('Value')
plt.ylabel('Frequency')
plt.title('Distribution with Anomalies (red lines)')

plt.tight_layout()
plt.show()

# 4. Advanced Deduplication
print("\n4. Advanced Deduplication (Record Linkage):")

# Sample duplicate records with variations
records = pd.DataFrame({
    'id': range(1, 11),
    'name': [
        'John Smith', 'John Smith', 'Jon Smith',  # Duplicates with typo
        'Jane Doe', 'Jane M. Doe', 'J. Doe',      # Same person, different formats
        'Bob Wilson', 'Robert Wilson',             # Nickname vs full name
        'Alice Johnson', 'Alice Jonson'            # Typo
    ],
    'email': [
        'john.smith@email.com', 'john.smith@email.com', 'j.smith@email.com',
        'jane.doe@email.com', 'jane.doe@email.com', 'jane.doe@email.com',
        'bob.w@email.com', 'robert.wilson@email.com',
        'alice.j@email.com', 'alice.j@email.com'
    ],
    'phone': [
        '555-1234', '555-1234', '555-1234',
        '555-5678', '555-5678', '555-5678',
        '555-9999', '555-9999',
        '555-1111', '555-1111'
    ]
})

print("   Original records:")
print(records)

def calculate_record_similarity(rec1, rec2):
    """Calculate similarity between two records"""
    name_sim = fuzzy_similarity(rec1['name'], rec2['name'])
    email_sim = 1.0 if rec1['email'] == rec2['email'] else 0.0
    phone_sim = 1.0 if rec1['phone'] == rec2['phone'] else 0.0
    
    # Weighted average
    return (name_sim * 0.5 + email_sim * 0.3 + phone_sim * 0.2)

def find_duplicate_groups(df, threshold=0.8):
    """Find groups of duplicate records"""
    groups = []
    processed = set()
    
    for i in range(len(df)):
        if i in processed:
            continue
        
        group = [i]
        for j in range(i + 1, len(df)):
            if j in processed:
                continue
            
            sim = calculate_record_similarity(df.iloc[i], df.iloc[j])
            if sim >= threshold:
                group.append(j)
                processed.add(j)
        
        if len(group) > 1:
            groups.append(group)
        processed.add(i)
    
    return groups

duplicate_groups = find_duplicate_groups(records, threshold=0.7)

print(f"\n   Found {len(duplicate_groups)} duplicate groups:")
for idx, group in enumerate(duplicate_groups, 1):
    print(f"\n   Group {idx}:")
    print(records.iloc[group][['name', 'email', 'phone']])

# 5. Automated Data Profiling
print("\n5. Automated Data Profiling:")

# Create comprehensive profile of dataset
def profile_dataframe(df):
    """Create comprehensive data quality profile"""
    profile = {
        'total_rows': len(df),
        'total_columns': len(df.columns),
        'memory_usage': df.memory_usage(deep=True).sum(),
        'column_profiles': {}
    }
    
    for col in df.columns:
        col_profile = {
            'dtype': str(df[col].dtype),
            'missing_count': df[col].isna().sum(),
            'missing_percentage': (df[col].isna().sum() / len(df)) * 100,
            'unique_count': df[col].nunique(),
            'cardinality': (df[col].nunique() / len(df)) * 100
        }
        
        # Numeric specific
        if pd.api.types.is_numeric_dtype(df[col]):
            col_profile.update({
                'min': df[col].min(),
                'max': df[col].max(),
                'mean': df[col].mean(),
                'median': df[col].median(),
                'std': df[col].std(),
                'zeros': (df[col] == 0).sum()
            })
        
        # String specific
        elif pd.api.types.is_string_dtype(df[col]) or df[col].dtype == 'object':
            col_profile.update({
                'max_length': df[col].astype(str).str.len().max(),
                'min_length': df[col].astype(str).str.len().min(),
                'avg_length': df[col].astype(str).str.len().mean()
            })
        
        profile['column_profiles'][col] = col_profile
    
    return profile

# Profile the imputation dataset
profile = profile_dataframe(df_impute)

print(f"   Dataset Profile:")
print(f"   Total rows: {profile['total_rows']}")
print(f"   Total columns: {profile['total_columns']}")
print(f"   Memory usage: {profile['memory_usage']:,} bytes")

print("\n   Column-level profiles:")
for col, col_prof in profile['column_profiles'].items():
    print(f"\n   {col}:")
    print(f"     Type: {col_prof['dtype']}")
    print(f"     Missing: {col_prof['missing_count']} ({col_prof['missing_percentage']:.1f}%)")
    print(f"     Unique: {col_prof['unique_count']} ({col_prof['cardinality']:.1f}%)")
    if 'mean' in col_prof:
        print(f"     Mean: {col_prof['mean']:.2f}, Median: {col_prof['median']:.2f}")
        print(f"     Range: [{col_prof['min']:.0f}, {col_prof['max']:.0f}]")

# 6. Constraint-Based Validation
print("\n6. Constraint-Based Validation (Complex Rules):")

# Sample data for validation
validation_data = pd.DataFrame({
    'customer_id': [1, 2, 3, 4, 5],
    'age': [25, 17, 45, 150, 30],
    'income': [50000, 30000, 80000, 20000, 60000],
    'credit_score': [700, 650, 800, 400, 720],
    'loan_amount': [200000, 50000, 300000, 10000, 150000],
    'employment_years': [5, 1, 20, -2, 8]
})

def validate_complex_rules(df):
    """Apply complex validation rules"""
    violations = []
    
    for idx, row in df.iterrows():
        row_violations = []
        
        # Rule 1: Age must be 18-100
        if row['age'] < 18 or row['age'] > 100:
            row_violations.append('Invalid age')
        
        # Rule 2: Credit score must be 300-850
        if row['credit_score'] < 300 or row['credit_score'] > 850:
            row_violations.append('Invalid credit score')
        
        # Rule 3: Loan amount should not exceed 5x annual income
        if row['loan_amount'] > row['income'] * 5:
            row_violations.append('Loan amount too high relative to income')
        
        # Rule 4: Employment years should be positive and less than age-16
        if row['employment_years'] < 0 or row['employment_years'] > (row['age'] - 16):
            row_violations.append('Invalid employment years')
        
        # Rule 5: Minimum income for certain loan amounts
        if row['loan_amount'] > 100000 and row['income'] < 40000:
            row_violations.append('Income too low for loan amount')
        
        violations.append({
            'customer_id': row['customer_id'],
            'violations': row_violations if row_violations else ['Valid']
        })
    
    return pd.DataFrame(violations)

validation_results = validate_complex_rules(validation_data)

print("   Validation Results:")
print(validation_results)

print("\n   Invalid records:")
invalid = validation_results[validation_results['violations'].apply(lambda x: x != ['Valid'])]
for _, row in invalid.iterrows():
    print(f"\n   Customer {row['customer_id']}:")
    for violation in row['violations']:
        print(f"     - {violation}")

# 7. Summary
print("\n7. Advanced Techniques Summary:")

summary = pd.DataFrame({
    'Technique': [
        'Fuzzy String Matching',
        'KNN Imputation',
        'Isolation Forest',
        'Record Linkage',
        'Data Profiling',
        'Constraint Validation'
    ],
    'Use_Case': [
        'Match similar text with typos',
        'Fill missing values intelligently',
        'Detect anomalies automatically',
        'Find duplicate records',
        'Assess data quality',
        'Enforce complex business rules'
    ],
    'Complexity': [
        'Medium',
        'Medium',
        'High',
        'High',
        'Low',
        'Medium'
    ],
    'When_to_Use': [
        'Name/address standardization',
        'Missing data with patterns',
        'Unknown outlier patterns',
        'Multiple data sources',
        'Data quality assessment',
        'Business rule enforcement'
    ]
})

print(summary.to_string(index=False))

print("\n   ✓ Advanced data cleaning techniques complete!")

## 15. Data Cleaning Pipeline & Best Practices

**What is it?**
A data cleaning pipeline is an automated, repeatable workflow that systematically applies cleaning operations to ensure consistent data quality.

**Why use it?**
- Ensures reproducibility
- Automates repetitive tasks
- Reduces human error
- Enables version control
- Scales to production
- Maintains consistency across datasets
- Facilitates collaboration

**When to use it?**
- Production environments
- Regular data updates
- Multiple data sources
- Team collaboration
- Need for reproducibility
- Quality assurance required

**Pipeline components:**
1. **Data validation**: Check input data quality
2. **Missing value handling**: Systematic imputation
3. **Type conversion**: Standardize data types
4. **Outlier treatment**: Detect and handle anomalies
5. **Standardization**: Normalize formats
6. **Deduplication**: Remove redundant records
7. **Quality checks**: Validate output
8. **Logging**: Track changes and issues

In [ ]:
# ============================================
# 15. DATA CLEANING PIPELINE & BEST PRACTICES
# ============================================

from datetime import datetime
import logging

print("=" * 80)
print("DATA CLEANING PIPELINE & BEST PRACTICES")
print("=" * 80)

# 1. Create logging system
print("\n1. Setting Up Logging:")

# Configure logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[
        logging.StreamHandler()
    ]
)
logger = logging.getLogger('DataCleaning')

print("   ✓ Logging system configured")

# 2. Define data cleaning pipeline class
print("\n2. Building Comprehensive Data Cleaning Pipeline:")

class DataCleaningPipeline:
    """
    Comprehensive data cleaning pipeline with logging and validation
    """
    
    def __init__(self, name="DataCleaningPipeline"):
        self.name = name
        self.logger = logging.getLogger(name)
        self.cleaning_report = {
            'timestamp': datetime.now(),
            'operations': [],
            'original_shape': None,
            'final_shape': None,
            'changes': {}
        }
    
    def log_operation(self, operation, details):
        """Log a cleaning operation"""
        self.logger.info(f"{operation}: {details}")
        self.cleaning_report['operations'].append({
            'operation': operation,
            'details': details,
            'timestamp': datetime.now()
        })
    
    def validate_input(self, df):
        """Validate input data"""
        self.log_operation("Input Validation", f"Shape: {df.shape}")
        self.cleaning_report['original_shape'] = df.shape
        
        # Check for empty dataframe
        if df.empty:
            raise ValueError("Input dataframe is empty")
        
        return df
    
    def handle_missing_values(self, df, strategy='median', threshold=0.5):
        """Handle missing values with multiple strategies"""
        missing_before = df.isna().sum().sum()
        
        # Drop columns with too many missing values
        missing_pct = df.isna().sum() / len(df)
        cols_to_drop = missing_pct[missing_pct > threshold].index.tolist()
        
        if cols_to_drop:
            df = df.drop(columns=cols_to_drop)
            self.log_operation("Drop Columns", f"Dropped {len(cols_to_drop)} columns with >{threshold*100}% missing")
        
        # Impute remaining missing values
        numeric_cols = df.select_dtypes(include=[np.number]).columns
        categorical_cols = df.select_dtypes(include=['object', 'category']).columns
        
        if strategy == 'median':
            df[numeric_cols] = df[numeric_cols].fillna(df[numeric_cols].median())
        elif strategy == 'mean':
            df[numeric_cols] = df[numeric_cols].fillna(df[numeric_cols].mean())
        
        df[categorical_cols] = df[categorical_cols].fillna(df[categorical_cols].mode().iloc[0])
        
        missing_after = df.isna().sum().sum()
        self.log_operation("Missing Values", f"Before: {missing_before}, After: {missing_after}")
        self.cleaning_report['changes']['missing_values'] = {
            'before': int(missing_before),
            'after': int(missing_after)
        }
        
        return df
    
    def remove_duplicates(self, df, subset=None):
        """Remove duplicate records"""
        duplicates_before = df.duplicated().sum()
        df = df.drop_duplicates(subset=subset)
        duplicates_after = df.duplicated().sum()
        
        self.log_operation("Remove Duplicates", f"Removed {duplicates_before} duplicates")
        self.cleaning_report['changes']['duplicates'] = {
            'removed': int(duplicates_before)
        }
        
        return df
    
    def handle_outliers(self, df, columns=None, method='iqr', factor=1.5):
        """Handle outliers using IQR or Z-score method"""
        if columns is None:
            columns = df.select_dtypes(include=[np.number]).columns
        
        outliers_count = 0
        
        for col in columns:
            if method == 'iqr':
                Q1 = df[col].quantile(0.25)
                Q3 = df[col].quantile(0.75)
                IQR = Q3 - Q1
                lower = Q1 - factor * IQR
                upper = Q3 + factor * IQR
                
                outliers = ((df[col] < lower) | (df[col] > upper)).sum()
                outliers_count += outliers
                
                # Cap outliers
                df[col] = df[col].clip(lower=lower, upper=upper)
        
        self.log_operation("Handle Outliers", f"Capped {outliers_count} outlier values")
        self.cleaning_report['changes']['outliers'] = {
            'capped': int(outliers_count)
        }
        
        return df
    
    def standardize_text(self, df, columns=None):
        """Standardize text columns"""
        if columns is None:
            columns = df.select_dtypes(include=['object']).columns
        
        for col in columns:
            # Strip whitespace
            df[col] = df[col].astype(str).str.strip()
            # Remove extra spaces
            df[col] = df[col].str.replace(r'\s+', ' ', regex=True)
            # Standardize case (title case)
            df[col] = df[col].str.title()
        
        self.log_operation("Standardize Text", f"Standardized {len(columns)} text columns")
        
        return df
    
    def convert_data_types(self, df, type_mapping=None):
        """Convert data types based on mapping"""
        if type_mapping:
            for col, dtype in type_mapping.items():
                if col in df.columns:
                    try:
                        df[col] = df[col].astype(dtype)
                        self.log_operation("Type Conversion", f"{col} → {dtype}")
                    except Exception as e:
                        self.logger.warning(f"Failed to convert {col} to {dtype}: {e}")
        
        return df
    
    def validate_output(self, df, rules=None):
        """Validate output data against rules"""
        issues = []
        
        # Check for remaining missing values
        missing = df.isna().sum().sum()
        if missing > 0:
            issues.append(f"Still has {missing} missing values")
        
        # Check for duplicates
        duplicates = df.duplicated().sum()
        if duplicates > 0:
            issues.append(f"Still has {duplicates} duplicates")
        
        # Custom rules
        if rules:
            for rule_name, rule_func in rules.items():
                if not rule_func(df):
                    issues.append(f"Failed rule: {rule_name}")
        
        if issues:
            self.logger.warning(f"Validation issues: {', '.join(issues)}")
        else:
            self.log_operation("Output Validation", "All checks passed ✓")
        
        self.cleaning_report['validation_issues'] = issues
        
        return df
    
    def clean(self, df, **kwargs):
        """Execute full cleaning pipeline"""
        self.logger.info(f"Starting {self.name}")
        
        # 1. Validate input
        df = self.validate_input(df)
        
        # 2. Handle missing values
        df = self.handle_missing_values(df, 
                                       strategy=kwargs.get('missing_strategy', 'median'),
                                       threshold=kwargs.get('missing_threshold', 0.5))
        
        # 3. Remove duplicates
        df = self.remove_duplicates(df, subset=kwargs.get('duplicate_subset'))
        
        # 4. Handle outliers
        df = self.handle_outliers(df, 
                                 columns=kwargs.get('outlier_columns'),
                                 method=kwargs.get('outlier_method', 'iqr'))
        
        # 5. Standardize text
        df = self.standardize_text(df, columns=kwargs.get('text_columns'))
        
        # 6. Convert data types
        df = self.convert_data_types(df, type_mapping=kwargs.get('type_mapping'))
        
        # 7. Validate output
        df = self.validate_output(df, rules=kwargs.get('validation_rules'))
        
        # 8. Finalize report
        self.cleaning_report['final_shape'] = df.shape
        self.logger.info(f"Completed {self.name}")
        
        return df
    
    def get_report(self):
        """Get cleaning report"""
        return self.cleaning_report

print("   ✓ DataCleaningPipeline class created")

# 3. Test the pipeline with sample data
print("\n3. Testing Data Cleaning Pipeline:")

# Create messy sample data
messy_data = {
    'customer_id': [1, 2, 2, 3, 4, 5, 6, 7, 8, 9, 10],  # Duplicate
    'name': ['  John Smith  ', 'jane DOE', 'jane DOE', 'Bob   Wilson', 
             'Alice Johnson', None, 'Charlie Brown', 'David Lee', 'Eva Martinez', 'Frank Zhang', 'Grace Kim'],
    'age': [25, 30, 30, 1000, 45, 35, np.nan, 50, 28, 32, 41],  # Outlier, missing
    'salary': [50000, 60000, 60000, 55000, 80000, np.nan, 70000, 90000, 52000, 58000, 75000],
    'email': ['john@email.com', 'jane@email.com', 'jane@email.com', 'bob@email.com',
             'alice@email.com', 'charlie@email.com', 'charlie@email.com', 
             'david@email.com', 'eva@email.com', 'frank@email.com', 'grace@email.com'],
    'city': ['New York', 'los angeles', 'los angeles', 'CHICAGO',
            'Boston', 'Seattle', 'Austin', 'Denver', 'Miami', 'Portland', 'Phoenix']
}

df_messy = pd.DataFrame(messy_data)

print("   Original messy data:")
print(df_messy)
print(f"\n   Shape: {df_messy.shape}")
print(f"   Missing values: {df_messy.isna().sum().sum()}")
print(f"   Duplicates: {df_messy.duplicated().sum()}")

# 4. Apply pipeline
print("\n4. Applying Cleaning Pipeline:")

pipeline = DataCleaningPipeline(name="CustomerDataCleaning")

df_cleaned = pipeline.clean(
    df_messy,
    missing_strategy='median',
    missing_threshold=0.8,
    duplicate_subset=['customer_id'],
    outlier_columns=['age', 'salary'],
    outlier_method='iqr',
    text_columns=['name', 'city', 'email']
)

print("\n   Cleaned data:")
print(df_cleaned)
print(f"\n   Shape: {df_cleaned.shape}")
print(f"   Missing values: {df_cleaned.isna().sum().sum()}")
print(f"   Duplicates: {df_cleaned.duplicated().sum()}")

# 5. Generate cleaning report
print("\n5. Cleaning Report:")

report = pipeline.get_report()
print(f"\n   Pipeline: {pipeline.name}")
print(f"   Timestamp: {report['timestamp']}")
print(f"   Original shape: {report['original_shape']}")
print(f"   Final shape: {report['final_shape']}")

print("\n   Operations performed:")
for op in report['operations']:
    print(f"     - {op['operation']}: {op['details']}")

print("\n   Changes summary:")
for change_type, change_data in report['changes'].items():
    print(f"     {change_type}: {change_data}")

# 6. Best Practices Summary
print("\n6. Data Cleaning Best Practices:")

best_practices = {
    '1. Document Everything': [
        'Keep detailed logs of all transformations',
        'Document assumptions and decisions',
        'Version control cleaning scripts'
    ],
    '2. Validate at Each Step': [
        'Check data quality before and after each operation',
        'Set up automated validation tests',
        'Monitor data distributions'
    ],
    '3. Make it Reproducible': [
        'Use pipelines and functions, not ad-hoc scripts',
        'Save intermediate states',
        'Use random seeds for reproducibility'
    ],
    '4. Handle Edge Cases': [
        'Test with extreme values',
        'Plan for new/unseen categories',
        'Handle null, empty, and special values'
    ],
    '5. Preserve Original Data': [
        'Never modify source data directly',
        'Create copies for transformations',
        'Keep backup of raw data'
    ],
    '6. Domain Knowledge': [
        'Understand business context',
        'Validate with domain experts',
        'Set realistic constraints'
    ],
    '7. Automate Quality Checks': [
        'Set up data quality metrics',
        'Create automated alerts',
        'Regular data profiling'
    ],
    '8. Performance Optimization': [
        'Use vectorized operations',
        'Process in chunks for large data',
        'Optimize memory usage'
    ]
}

for practice, guidelines in best_practices.items():
    print(f"\n   {practice}:")
    for guideline in guidelines:
        print(f"     • {guideline}")

# 7. Common Pitfalls to Avoid
print("\n7. Common Data Cleaning Pitfalls to Avoid:")

pitfalls = pd.DataFrame({
    'Pitfall': [
        'Data Leakage',
        'Overfitting to Noise',
        'Ignoring Duplicates',
        'Arbitrary Outlier Removal',
        'Inconsistent Transformations',
        'Not Validating Assumptions',
        'Losing Information',
        'Ignoring Time Dependencies'
    ],
    'Impact': [
        'Biased model performance',
        'Poor generalization',
        'Inflated statistics',
        'Loss of valuable data',
        'Unreliable results',
        'Incorrect conclusions',
        'Reduced model accuracy',
        'Invalid analysis'
    ],
    'Solution': [
        'Clean train/test separately',
        'Keep original data, validate changes',
        'Always check for duplicates',
        'Use statistical methods, justify removal',
        'Use pipelines, maintain consistency',
        'Test assumptions, validate with experts',
        'Document what was removed and why',
        'Preserve temporal ordering'
    ]
})

print(pitfalls.to_string(index=False))

# 8. Recommended Tools and Libraries
print("\n8. Recommended Python Libraries for Data Cleaning:")

libraries = pd.DataFrame({
    'Library': [
        'Pandas',
        'NumPy',
        'scikit-learn',
        'Great Expectations',
        'pandas-profiling',
        'fuzzywuzzy',
        'pyjanitor',
        'datacleaner'
    ],
    'Purpose': [
        'Core data manipulation',
        'Numerical operations',
        'Preprocessing, imputation, scaling',
        'Data validation framework',
        'Automated EDA and profiling',
        'Fuzzy string matching',
        'Clean API for cleaning operations',
        'Automated cleaning'
    ],
    'Use_When': [
        'Always (fundamental)',
        'Numerical computations',
        'ML pipelines, transformations',
        'Production data validation',
        'Initial data exploration',
        'Text standardization',
        'Method chaining workflows',
        'Quick automated cleaning'
    ]
})

print(libraries.to_string(index=False))

# 9. Final Recommendations
print("\n9. Final Recommendations:")

print("""
   Data Cleaning Workflow:
   
   1. UNDERSTAND → Explore and profile your data
   2. PLAN → Define cleaning strategy and rules
   3. VALIDATE → Check input data quality
   4. CLEAN → Apply systematic transformations
   5. VERIFY → Validate output meets requirements
   6. DOCUMENT → Log all changes and decisions
   7. ITERATE → Refine based on results
   8. AUTOMATE → Build reusable pipelines
   
   Remember:
   • Data cleaning is iterative, not one-time
   • Domain knowledge is as important as technical skills
   • Good documentation prevents future headaches
   • Automation ensures consistency and saves time
   • Always validate assumptions with data and experts
   • Clean data is the foundation of good analysis
""")

print("=" * 80)
print("CONGRATULATIONS! DATA CLEANING LEARNING COMPLETE")
print("=" * 80)

print("""
You've completed the comprehensive Data Cleaning guide! You now know:

✓ How to handle missing values (12+ strategies)
✓ Detecting and removing duplicates (exact, partial, fuzzy)
✓ Data type conversion and optimization
✓ String cleaning and standardization
✓ Outlier detection and handling (6+ methods)
✓ Data validation (types, ranges, formats, business rules)
✓ Handling inconsistent data across sources
✓ Date & time cleaning (parsing, validation, extraction)
✓ Categorical data cleaning (encoding, grouping, standardization)
✓ Numerical data cleaning (units, precision, binning)
✓ Text data cleaning (NLP preprocessing)
✓ Standardization & normalization (6+ techniques)
✓ Advanced techniques (fuzzy matching, ML imputation, anomaly detection)
✓ Building production-ready cleaning pipelines
✓ Best practices and common pitfalls

Next Steps:
• Practice with real-world messy datasets
• Build cleaning pipelines for your projects
• Learn about data quality frameworks (Great Expectations)
• Study feature engineering for machine learning
• Explore automated ML pipelines (MLOps)

Happy Data Cleaning! 🧹✨
""")